# Inputs

In [48]:
## Step 1: upload files (to be loaded into script) into session.
## Step 2: Assign file names to appropriate variables so that script know which files to associatee with each step of process.
fn_assets <- "b1. P08'F20 SMKT FAR BOOK detailed Z056 all SWAN Classes-Adds only.csv" # expects a csv
fn_ProjectNames_removeDuplicates <- 'ProjectNames_removeDuplicates.csv' # expects a csv

data_deactivated_asset_files <- c('F19 BOOK Disposal check AUD Co codes 1000 project SWAN list.xlsx', # expects a xlsx files
'F19 BOOK Disposal check AUD Co codes 1010-1999 project SWAN list.xlsx',
'F19 but F20 BOOK Disposal check AUD Co codes 1000-1910 in QTRS SWAN to 26.11.19 .xlsx')

# storeList_and_allocationRate from https://docs.google.com/spreadsheets/d/1pbuJ84AVMzRxIdL2_Zmb8JwbFOF5HeU5Bst63Z4r2xc/edit#gid=1080049251
storeList_and_allocationRate <- "Store List & Allocation Rate - Jan20.xlsx" # expects a xlsx file
storeList_and_allocationRateRange <- "3:562" # this is the data range excl headers. e.g. if data is row 3 to row 561 in storeList_and_allocationRate file then this should be '3:561'

## Step 3: declare variables.
fiscal_year_ <- 'F20' # E.g. 'F20'

## Step 4: Run the script (Ctrl+F9)

## Step 5: If script completes successfully download zipped_all_files and save (as well as a copy of this notebook via 'make a copy' in drive) to a new google drive folder so that each run is stored.
## note that a file called 'env.RData' is saved (and zipped). This has just saved the current workspace. Can come in helpful in case you need to get back to same workspace in the future, e.g. to interrogate an issue, without needing to re-run entire script.


# Checks

In [49]:
######### Check that all the files that should exists do exist

lFiles <- list.files()
for(f in c(fn_assets, fn_ProjectNames_removeDuplicates, data_deactivated_asset_files, storeList_and_allocationRate)){
  if(!(f %in% lFiles)){
    print(paste0("file ", f, " expected but not found."))
    stop('script ended')
  }
}

## check fn_assets file column names match expectation
for(cn in c('Asset', 'SN', 'Asset description', 'CoCode', 'Clas', 'APC FY start', 'Dep. FY start', 'Bk.val.FY strt', 'Acquisition', 'Dep. for 8MTHS', 'Retirement', 'Dep.retir.', 'IntraCoy Trf.', 'IntraCoy Dep.', 'InterCoy Trf.', 'InterCoy Dep.', 'Reval (Depn)', 'Reval (APC)', 'Current APC', 'Accumul. dep.', 'Curr.bk.val.', 'Cap.date', 'Cost Ct', 'Sub', 'WBS element')){
  if(!(cn %in% colnames(data.table::fread(fn_assets)))){
    print(paste0("Column ", cn, " not in ", fn_assets))
    stop('script ended')
  }
}


# Run process

In [50]:
########## Script_Final_00.R

#===============================================================================================
#IMPORT LIBABRIES

install.packages('reshape2'); library(reshape2)
library(ggplot2)
#library(bigrquery)
library(readr)
library(dplyr)
library(ggplot2)
#library(pander)
library(scales)
library(stringr)
library(readxl)


#===============================================================================================
#CREATE ASSET DATA SET 
#setwd("U:\\Projects\\Backup_C_Projects_20191021\\ED - Swan Data\\20200313_0831_AssetLoad\\DataFromJason/")

DataSet_Assets_01  <- data.table::fread(fn_assets) %>%
  .[,1:25] %>%
  dplyr::mutate(Year = fiscal_year_)

FileData_ALL <- DataSet_Assets_01

# pc_to_run - if it contains profit centres rest of script will oly look at those PC's. If NA rest of script will run on ALL data.
pc_to_run <- NA # c('1129','1132','1199','1205','1368','3354','4304','4324','4339','4600')

if(!is.na(pc_to_run) %>% all()){
  FileData_ALL <- FileData_ALL %>%
    filter(`Cost Ct` %in% pc_to_run)
}


rm(DataSet_Assets_01)

# clean column names
colnames(FileData_ALL) <- gsub("\\s+|\\.","_", colnames(FileData_ALL)) %>%
  gsub("\\(|\\)","",.) %>%
  gsub("_$","",.) %>%
  gsub("[_]+","_",.)


#===============================================================================================
#CREATE PROJECT NAMEDATA SET 

DataSet_ProjectNames <- data.table::fread(fn_ProjectNames_removeDuplicates)
DataSet_ProjectNames$ProjectDescription <- ''
DataSet_ProjectNames$cleanWBSKey <- ''
DataSet_ProjectNames$ProjectDescription <- paste(DataSet_ProjectNames$TXTMD,DataSet_ProjectNames$Field4,DataSet_ProjectNames$Field5,DataSet_ProjectNames$Field6)
DataSet_ProjectNames$cleanWBSKey <- gsub("\\.|[0]+$","",DataSet_ProjectNames$WBS_ELEMT)

#===============================================================================================
#CREATE CLEAN DATA SET FOR ANALYSIS SET
# all NA's
mclean_numeric_fields <- function(x){
  gsub("- |[a-z]|^-$|,|\\s+", '', x) %>%
    gsub("\\s+", '', .) %>%
    gsub("^-$", '', .) %>%
    as.numeric(.)
}
options(scipen=9999999) # to avoid scientific notation e.g. 2+e5 being converted to NA once mclean_numeric_fields() is run
FileData_ALL <- FileData_ALL %>%
  dplyr::mutate(APC_FY_start = mclean_numeric_fields(APC_FY_start)
                , Curr_bk_val = mclean_numeric_fields(Curr_bk_val)
                , Dep_FY_start = mclean_numeric_fields(Dep_FY_start)
                , Bk_val_FY_strt = mclean_numeric_fields(Bk_val_FY_strt)
                , Acquisition = mclean_numeric_fields(Acquisition)
                , Dep_for_8MTHS = mclean_numeric_fields(Dep_for_8MTHS) # in this file 'Dep_for_year' is called 'Dep_for_6MTHS'
                , Retirement = mclean_numeric_fields(Retirement)
                , Dep_retir = mclean_numeric_fields(Dep_retir)
                , IntraCoy_Trf = mclean_numeric_fields(IntraCoy_Trf)
                , IntraCoy_Dep = mclean_numeric_fields(IntraCoy_Dep)
                , InterCoy_Trf = mclean_numeric_fields(InterCoy_Trf)
                , InterCoy_Dep = mclean_numeric_fields(InterCoy_Dep)
                , Reval_Depn = mclean_numeric_fields(Reval_Depn)
                , Reval_APC = mclean_numeric_fields(Reval_APC)
                , Current_APC = mclean_numeric_fields(Current_APC)
                , Accumul_dep = mclean_numeric_fields(Accumul_dep)
                , cleanWBSKey = gsub("\\.|[0]+$", "", WBS_element)
  )

#LOOKUP THE PROJECT DESCRIPTIONS
#===============================================================================================
c1 <- nrow(FileData_ALL)
FileData_ALL <- FileData_ALL %>%
  dplyr::left_join(
    DataSet_ProjectNames %>%
      dplyr::select(cleanWBSKey, ProjectDescription)
    , by="cleanWBSKey"
  ) %>%
  mutate(ProjectDescription = ifelse(cleanWBSKey == '','',ProjectDescription))
c2 <- nrow(FileData_ALL)
# check didn't join on 1 to many:
(c1 == c2)

rm(DataSet_ProjectNames)

FileData_ALL$Rule_BWS <- ''
FileData_ALL$Rule_Food <- ''
FileData_ALL$Rule_Shared <- ''
FileData_ALL$Rule_FormatType <- ifelse(FileData_ALL$WBS_element=='0','WBS Zero','Project')
FileData_ALL$Rule_Final <- ''

# IT rule first
IT_2_match <- c(
  '^H\\.1861'
  , '^H\\.1809'
  , '^H\\.1935'
  , '^H\\.2959'
  , '^H\\.1789'
  , '^H\\.3727'
  , '^H\\.2506'
  , '^H\\.3678'
  , '^H\\.2427'
  , '^H\\.1928'
  , '^H\\.3452'
  , '^H\\.3871'
  , '^H\\.3873'
  , '^H\\.3772'
  , '^H\\.2162'
  , '^H\\.1175'
  , '^H\\.1248'
  , '^H\\.2401'
  , '^H\\.3726'
  , '^H\\.4008'
  , '^H\\.1927'
  , '^H\\.3172'
  , '^H\\.3171'
  , '^H\\.3100'
  , '^H\\.3278'
  , '^H\\.1258'
  , '^H\\.3268'
  , '^H\\.3271'
  , '^H\\.2160'
  , '^H\\.4021'
  , '^H\\.4012'
  , '^H\\.0772'
  , '^H\\.1955'
  , '^H\\.1605'
  , '^H\\.1847'
  , '^H\\.3335'
  , '^H\\.0481'
  , '^H\\.1376'
  , '^H\\.1763'
  , '^H\\.2884'
  , '^H\\.1188'
  , '^H\\.2130'
  , '^H\\.3701'
  , '^H\\.0583'
  , '^H\\.1168'
  , '^H\\.1389'
  , '^H\\.1547'
  , '^H\\.2464'
  , '^H\\.2537'
  )
IT_2_match_piped <- paste0(IT_2_match, collapse="|")

FileData_ALL$Rule_Food[grep(IT_2_match_piped, FileData_ALL$WBS_element)]  <- 'Food'

#APPLY RULES TO THE DATA 
#==========================================================================
#Rule 1.1 - WBS starting with 'K6' = BWS
FileData_ALL$Rule_BWS[intersect(
  grep("^K6", FileData_ALL$WBS_element)
  , which(FileData_ALL$Rule_BWS == '' & FileData_ALL$Rule_Food == '')
  )]  <- 'BWS'

#===============================================================================================
#Rule 2.1 - WBS starting with R = Renewal
#Rule 2.2 - WBS starting with E = Extention
#Rule 2.3 - WBS starting with N = New

FileData_ALL$Rule_FormatType[grep("^R", FileData_ALL$WBS_element,ignore.case = TRUE)]  <- 'Renewal'
FileData_ALL$Rule_FormatType[grep("^E", FileData_ALL$WBS_element,ignore.case = TRUE)]  <- 'Extention'
FileData_ALL$Rule_FormatType[grep("^N", FileData_ALL$WBS_element,ignore.case = TRUE)]  <- 'New'

#===============================================================================================
#Rule 3.1 - WBS ending ".081" = BWS 
#Rule 3.2 - WBS ending ".011" =  Food
#Rule 3.3 - WBS ending ".021" =  Food
#Rule 3.4 - WBS ending ".041" =  Food
#Rule 3.5 - WBS ending ".051" =  Food
#Rule 3.6 - WBS ending ".061" =  Food
#Rule 3.7 - WBS ending ".071" =  Food
#Rule 3.8 - WBS ending ".031" =  Food

FileData_ALL$Rule_BWS[intersect(
  grep("\\.081$", FileData_ALL$WBS_element)
  , which(FileData_ALL$Rule_BWS == '' & FileData_ALL$Rule_Food == '')
  )]  <- 'BWS'
FileData_ALL$Rule_Food[intersect(
  grep("\\.011$|\\.021$|\\.031$|\\.041$|\\.051$|\\.061$|\\.071$", FileData_ALL$WBS_element)
  , which(FileData_ALL$Rule_BWS == '' & FileData_ALL$Rule_Food == '')
  )]  <- 'Food'

#===============================================================================================
#Rule 4.1 - Project Description contains  'FSL','BWS','Liq' and 'Beer' = BWS (ignore case) 
#Rule 4.2 - Asset Description contains  'FSL','BWS','Liq' and 'Beer' = BWS (ignore case)

#FileData_ALL$Rule_BWS[grep("fsl|bws|liq|beer", FileData_ALL$ProjectDescription, ignore.case = TRUE)]  <- 'BWS'
#FileData_ALL$Rule_BWS[grep("fsl|bws|liq|beer", FileData_ALL$Asset_description, ignore.case = TRUE)]  <- 'BWS'


for(x in "fsl|bws|liq|beer") {
  which_no_rule_yet <- which(FileData_ALL$Rule_Food == '' & FileData_ALL$Rule_BWS == '')
  if(length(which_no_rule_yet)<1){break}
  which_match <- grep(x, FileData_ALL$ProjectDescription[which_no_rule_yet], ignore.case = TRUE)
  idx_match <- which_no_rule_yet[which_match]
  FileData_ALL$Rule_BWS[idx_match]  <- 'BWS'
}

for(x in "fsl|bws|liq|beer") {
  which_no_rule_yet <- which(FileData_ALL$Rule_Food == '' & FileData_ALL$Rule_BWS == '')
  if(length(which_no_rule_yet)<1){break}
  which_match <- grep(x, FileData_ALL$Asset_description[which_no_rule_yet], ignore.case = TRUE)
  idx_match <- which_no_rule_yet[which_match]
  FileData_ALL$Rule_BWS[idx_match]  <- 'BWS'
}

#===============================================================================================
#Rule 5.1 - Asset Descriptions that contains Food Keywords  = Food
#Rule 5.2 - Project Descriptions that contains Food Keywords  = Food
FOODKeyWords <- c("100% Click Collect","Bakery","Chicken","Dairy","Deli","Fish","Frozen","Meat","Produce","Pizza","Bread","cooktop","Hair","Covergirl","Magazine","Maybelline","Razor","Pillow","Book","Kids","Pizza","Bake","Grocery","Baby","Cereal","FRUIT","Stationery","Coffee","Dog","Milk","Snack","Spice","Cosmetic","Cleaner","Banana","Herb","Rimmel","SallyHansen","Garnier","L'Oreal","Olay","Nivea","Chick","Gift Card","Pasta","Sugar","Biscuit","Medicinal","Dinner Made Easy","Long Life","Freezer","Weighscales","Fresh","Newspaper","Slicer","Market Buy","HBA","HBB","Cig","Tobacco","Trolley","Promotional Ends","Online equipment","Home","lettuce","mushrm","mushroom","leaf","organic","chilli","Hygiene","hygeine","Basket","Chiller","oven","retarder","bky","berry","Gate Checkout","Confect","Wellness","D-Pallets","C n C","CnC","pastry","odd bunch","skin","veg","berri","balustrade","floral","cooker","Meal","flower","pick up","velocity","maxi ba","online","umbrella","misting","footwear","nuts","baking","jelly","children","loaf","platter","dough","aircare","seafood","Clean&Clear","shampoo","Condiment","Sushi","Spices","Gift","Cigarette","Cheese","Digital Media","Scorecard","health","de-pallet","Food","Self Scanning checkout","SCO","Self Serve Express Light","Self Serve","Self Checkout","checkout","Self-Scan","Family","laundry","household","hosiery","noodle","animal","beauty","perishable","poultry","toothbrush","tenderiser","Sun&Skin","My kitchen","cooking","display broom","beef","salon","skincare","clothes","styling","telco","apple","corn","orange","lemon","pumpkin","tomato","petrol","service desk","servdesk","servdsk","toilet paper","ladies","dental","Mens grooming","mouth wash","Macro Plain","Skin/Sun","deodorant","photo","booth","dock","stockroom","Gondola","coolroom","macro space","S4S","plantroom","oralcare","floss","Travel & Convenience","Collapsible","Express Mag","Mag Express","pear","Fascia","Facia","kiosk","Merch Mobile","Merchandise","Mesh Capping","Scale","Supergrip","merch","Neutro","garn","dove","vaseline","Hand&Body","Clearasil","macro","Hand Wash","travelpacks","kronos","Self Scan")
FOODKeyWords_piped <- paste(FOODKeyWords,collapse = "|")

#FileData_ALL$Rule_Food[grep(FOODKeyWords_piped, FileData_ALL$Asset_description, ignore.case = TRUE)]  <- 'Food'
#FileData_ALL$Rule_Food[grep(FOODKeyWords_piped, FileData_ALL$ProjectDescription, ignore.case = TRUE)]  <- 'Food'

# create function that accepts a vector of strings to grep only on records that don't match a rule
for(x in FOODKeyWords) {
  which_no_rule_yet <- which(FileData_ALL$Rule_Food == '' & FileData_ALL$Rule_BWS == '')
  if(length(which_no_rule_yet)<1){break}
  which_match <- grep(x, FileData_ALL$Asset_description[which_no_rule_yet], ignore.case = TRUE)
  idx_match <- which_no_rule_yet[which_match]
  FileData_ALL$Rule_Food[idx_match]  <- 'Food'
}

for(x in FOODKeyWords) {
  which_no_rule_yet <- which(FileData_ALL$Rule_Food == '' & FileData_ALL$Rule_BWS == '')
  if(length(which_no_rule_yet)<1){break}
  which_match <- grep(x, FileData_ALL$ProjectDescription[which_no_rule_yet], ignore.case = TRUE)
  idx_match <- which_no_rule_yet[which_match]
  FileData_ALL$Rule_Food[idx_match]  <- 'Food'
}

#Rule 6.1 - Asset Descriptions that contains specified list of Food Materials  = Food
#===============================================================================================

MaterialMaster_FOOD_LIST <- list(
  MaterialMaster_FOOD_001_100 = c("Wrapper Console on stand - 24 Volt","Stand Dishwasher TW","Temperature Kit PropaTemp Large","Temperature Kit PropaTemp Small","Wrapper Bench Type - 24 Volt","Kit Visual Merchandiser Small \\(DD\\)","Kit Visual Merchandiser Medium \\(DD\\)","Kit Visual Merchandiser Large \\(DD\\)","Kit Visual Merchandiser Small \\(Mills\\)","Kit Visual Merchandiser Medium \\(Mills\\)","Kit Visual Merchandiser Large \\(Mills\\)","Training on Equipment for Staff","Install & Commission MeatWrap Add Charge","Test Weight Scales 500gm","Knife Swibo Boning 160mm","Board Cutting White 610x450x13mm","Bin Round Stainless Steel","Bin Blue 40Litre","Bin Red 40Litre","Bin Orange /Lifta Lid 120Litre","Bin 240L Orange w/- lifta lid","Bin Lime Lide 240Ltr","Board Cutting White 300x205x13mm","Tray Polycarb Half Market Blk 360x203x56","Tray Polycarb Qtr High Black 180x304x56","Tray Polycarb Third High Blk 240x304x56","Tray Polycarb Half High Blk 360x304x56","Tray Polycarb Full Low Blk 720x304x31","Dispenser only for Plastic Bags","Clip with Large Card Holder Cheese Case","Ticket Holder Clear 100mm with 85mm base","Holder Ticket A4","BinWheelie240LimeGrnLidF/RecyleW/LidLift","BinWheelie120RedLidG/WasteL/fillW/L.Lift","BinWheelie240RedLidG/WasteL/fillW/L.Lift","BinWheelie 120 RedLid Gen.Waste Landfill","BinWheelie120L LimeGrnLid FoodRecycling","BinWheelie120BlueLid CardboardRecycling","BinWheelie80LLimeGrnLidF/RecyleW/L.Lift","Dispenser Hairnet","Spoon S\\S Blk Plastic Handle Solid 330mm","Hook Single PK100 Gantry Sign SemiTD","Hook Double PK100 Gantry Sign SemiTD","Hook GA Gantry Hanging Sign PK10 SemiTD","Hook Spacer Gantry Sign PK50 SemiTD","Tong 300mm S/Steel w/Black PVC Handle","Stand Chrome For 30Lt Hand Baskets","Ticket Holder Snapshut Clear","Data Strip Flipscan 39x75mm Black","Data Strip Flat 1100mm Black","Data Strip Flat 1100mm White Satin","Data Strip Mid Shelf 915","Data Strip 510 Base Shelf","End Cap LH and RH pair","Data Strip Flat 39mm Black 914mm","Data Strip Hinged 39mm Black 914mm","Data Strip Flipscan 39x75mm White","Data StripFlipper39mm White cut to 435mm","Dispenser for 606 Cleaning Cloth","ContainerStorage 18.9L w/Lid 457x305x229","ContainerStorage 13.2L w/Lid 457x305x152","Data Strip Flat Provista Universal White","D/Strip Upslope Provista Universal White","Data Strip Flat 914mm Black Pack of 50","Data Strip Flat Provista 1220mm Black","Data Strip Upslope Provista 1220mm Black","Data Strip T-Bar 914mm White Satin","Data Strip Angled T-Bar 914mm W/Satin","Data Strip Flat 914mm White Satin","Data Strip BEM 550mm","Data Strip Flat 914mm White Satin","Data Strip Upslope 914mm Black","Data Strip T-Bar 914mm White Satin","Data strip flat 39mm Clear 914mm","Standard Black GTrack","Joining Clip Universal GTrack & MTrack","Kit Arm Steet Sign Adjustable GTrack","Bracket L Magnetic GTrack","Bracket L Magnetic Extension GTrack","Bracket Z Flat Wall GTrack","Switch Surround GTrack","Cap Corner GTrack","Cap End Universal GTrack & MTrack","Standard Black GTrack w/Mag Tape MTrack","Cap Corner \\(pair\\) MTrack","Beam Cross OnCabinet","Column End OnCabinet","Column Middle OnCabinet","Arm Street Sign OnCainet","Bracket Magnetic Promotional OnCabinet","End Frame OnCabinet","Joiner End Frame to End Column OnCabinet","Holder Ticket Trangular 4UP OnCabinet","Frame One Piece - On Cabinet GTrack","Frame Joiner - On Cabinet GTrack","Street Sign Adaptive - On Cabinet GTrack","Rail Hang & Components 1650x420","KJ09 - Cladding Cheese Island 2500mm","KJ09 - Cladding Cheese Island 3750mm","KJ10 - Cladding Cheese Island 2500mm")
  , MaterialMaster_FOOD_101_200 = c("KJ10 - Cladding Cheese Island 3750mm","KH14 - Cladding Chicken Warmer 1575mm","KH14 - Cladding Chicken Warmer 2200mm","Frame 2002 for 535mm P/Point Telescopic","Frame 4 Way Price Point 365mm","Sign Safe Working Load","Trolley Stock for Nally Tub","Trolley Stainless Steel for Trays","Bench Stainless Steel for Bakery Scale","Installation Pallet Racking per Strt Bay","Installation Pallet Racking per Add Bay","Holder for Knives - Safety","Racking Coolroom 1200X525","Racking Coolroom 600X525","Racking Coolroom 1200X600","Racking Coolroom 600X600","Racking Coolroom A4-Safe Working Loads","PostClamp Racking Coolroom EQAA8105","Racking Coolroom 1350x600 Meat","Hose Reel S/SteelRetractible Wall Mount","Hose Reel S/SteelRetractible Floor Mount","Kit C/RoomRack 4 Tier 610x1380x1830 Strt","Kit C/RoomRack 4 Tier 610x1380x1830 Add","Coolroom Racking Metro MaxQ Kit 1220mm","Coolroom Racking Metro MaxQ Kit 610mm","Coolroom Shelving Metro 1065x455mm","Coolroom Shelving Metro 915x535mm","Insect Control Unit Sectarid / Glueboard","Insect Control Unit Luralite","Kit C/RoomRacking 4 Tier 1220x610mmStart","Kit C/RoomRacking 4 Tier 1220x610mm Add","Kit C/RoomRacking 4 Tier 1220x460mm Star","Kit C/RoomRacking 4 Tier 1220x460mm Add","Kit C/RoomRacking 4 Tier 760x610mm Start","Kit C/RoomRacking 4 Tier 760x610mm Add","Kit C/RoomRacking 4 Tier 760x460mm Start","Kit C/RoomRacking 4 Tier 760x460mm Add","Corner Connector per Shelf","Rail for Label Dispenser","Chiller Water & Meter Kit Stoppil","Proover Retarder 4 Door Salva","Mixer Spiral MP100 Escher","Proover Retarder 3 Door Salva","Moulder Dough Merand","Divider Rounder Fortuna Model 4 w/Wheels","Moulder Versatile 2 W/Light Speed Guard","Slicer Bread Twin Curl Flow","Scale Hanging 10 Kg with Scoop & Cradle","Scale Ingredient Electronic","Panel Rear for Island Salva Oven","Induction Hot Plate Incl. 2 S/S Pots","Printer Label Bakery","Salva Sirocco Electric Rack Oven","Salva NXE 12 Tray Deck Oven","Slicer Bread Mobile with Stand","Bowl S/S Hobart HL600 Legacy 60Qrt Mixer","Bowl Truck Hbrt HL600 Legacy 60Qrt Mixer","Oven Parbake Double Stack SALVA W/Stand","Mixer Complete Planetary 40 Ltr TK40","Beater 40 Ltr - Mixer Planetary TK40","Beater 20 Ltr - Mixer Planetary TK40","Whisk 40 Ltr - Mixer Planetary","Whisk 20 Ltr - Mixer Planetary TK40","Bowl S/S 40 Ltr - Mixer Planetary TK40","Bowl S/S 20 Ltr - Mixer Planetary TK40","Trolley Bowl 40 Ltr Mixer Planetary TK40","SlicerBreadBenchtop20mmBladeThick Escher","Oven Rotel 3 12 Tray Deck Oven","Oven Rotel 3 24 Tray","Fryer Donut Belshaw Robot MK2 w/Brackets","Fryer Donut Belshaw Mini Hopper","Machine Donut On Bench W/Hood","Machine Donut On Bench NO Hood","Proover Retarder 2 Door Salva","Proover Retarder 2 door Pani","Beater 10 Ltr - Mixer Planetary TK20","Beater 20 Lt r - Mixer Planetary TK20","Bowl S/S 10 Ltr - Mixer Planetary TK20","Bowl S/S 20 Ltr - Mixer Planetary TK20","Mixer Planetary Complete TK20","Whisk 10 Ltr - Mixer Planetary TK20","Whisk 20 Ltr - Mixer Planetary TK20","Scale Ingredient Electronic with Scoop","Rack S/S for Revent Oven 9 Tray","Rack S/S for Revent Oven 15 Tray","Oven Salva 5+9 OnStand W/Hood &FlueKit","Tray Kit For Salva Oven","Rack 9 Shelf for Salva Sirocco Oven","Rack 15 Shelf for Salva Sirocco Oven","Bag Closer Semi Auto","Bowl S/S for Hobart 20 Qrt Mixer","Beater for Hobart 20 Qrt Mixer","Whip Heavy Duty for Hobart 20 Qrt Mixer","Hood Salva Rack Oven","Hood Salva Deck Oven","Freezer Bakery 2 Door Upright","Microwave M/Master 1400x425x548x335","Divider Square with grid attachment","Divider Square with grid attachment","Kit Stone Rotel 3 Oven")
  , MaterialMaster_FOOD_201_300 = c("Kit Stone Rotel 2 Oven","Kit Baguette Grid","Grid 2x4","Grid 6x5 Rectangle","Grid 4x5","Kit Baguette Grid","Grid 2x4","Grid 6x5 Rectangle","Grid 4x5","Hood Eyebrow Rotel 3","Proover Retarder 4 Door Pani","Proover Retarder 3 Door Pani","Mixer Spiral 80kg with S/S Bowl Cover","Divider Bun Rounder - Semi Automaic","Chiller Water & Meter Kit","Oven Rack Taglivani Electric","Mixer Complete Planatary 40qrt PL40N","Bowl S/S 40qrt - Mixer Planetary PL40N","Whisk 40qrt - Mixer Planetary PL40N","Beater 40qrt - Mixer Planetary PL40N","Bowl S/S 20qrt - Mixer Planetary PL40N","Whisk 20qrt - Mixer Planetary PL40N","Beater 20qrt - Mixer Planetary PL40N","Mixer Complete Planatary TK20","Bowl S/S 20qrt - Mixer Planetary TK20","Whisk 20qrt - Mixer Planetary TK20","Beater 20qrt - Mixer Planetary TK20","Bowl S/S 10qrt - Mixer Planetary TK20","Whisk 10qrt - Mixer Planetary TK20","Beater 10qrt - Mixer Planetary TK20","Rack 9 Shelf for Taglivani Oven","Rack 15 Shelf for Taglivani Oven","Trolley Bowl 40Ltr Mixer Planetary PL40N","Oven Mit/Glove Single Elbow Length Oates","Oven Mits Double Pocket Oates","Covers 12 Tray Rack Mini Bake 1660x580","Opener Can 36cm Bonzer Bench Type","Knife Swibo Wavey Edge 250mm","Knife Swibo Wavey Edge 300mm","Pot Boiling Aluminium 200x310mm 14Ltr","Bin Disposal Plastic with Lid 55 Litre","Tray Baking Three Sided","Tray Baking Four sided","Tin Sponge Cake 178mm","Tin Bread Pan 840g set of 2","StripRubberWH300x9x1400ReplaceF/EQBB8068","Pan Bread 680g set of 3","Wire Cooling Straight","Pan Bread Vienna set of 3","Tray French Stick 5 Impression","Bin Nally No 15 White with Lid and Dolly","Pan Bread 800g set of 2 w/drop on lid","Mat Bakery Happy Grip Top 600mmx900mm","Rattan Basket Plastic 700x360x395mm","Rattan Basket Plastic 700x360x285mm","Rattan Basket Plastic 700x360x175mm","RattanBasket 5ft Prop Bread 843x360x395","RattanBasket 5ft Prop Bread 843x360x285","RattanBasket 5ft Prop Bread 843x360x175","Kit Brush Maxi Bakery","Kit Brush Bakery","Tray Lamington","Cake Spinner","Tray Inserts Lamington 200x402","Tray Inserts Lamington 370x402","Tray Inserts Lamington 550x402","Tray 3 Sided Long Open Rotel Naked","Tray Baking Perforated","Tray Muffin","Knife Swibo Dough","Knife Swibo Vienna 13cm Serrated","Siphon for EQBB1109 Donut Machine","Kit Small Tools Bakery","Sheets Teflon for Grid Divider Kit9 Vanr","Racks Dough Tub for Grid Divider Vanrooy","DoughTub incl lids forGridDivider VanRoo","Grid Divider 2x3","Grid Divider 2x3","ContainerStorage 5.7L w/Lid 222x222x186","ContainerStorage 3.8L w/Lid 181x181x185","Container 1.5ltr P/Carb100mmCambro \\(PK6\\)","Lid for 1.5ltr Container Cambro \\(PK6\\)","Container5.9ltr P/Carb 100mmCambro \\(PK6\\)","Lid for 5.9ltr Container Cambro \\(PK6\\)","Basket Polywicker 300x400x40","Train Bakery Display Unit With Shelf","Artisan Bread Unit","Bakery Train Table","Bakery Train Base Unit","Bakery Train Set","Column Unit","Impulse Table","Deli Top Display","Tall Wall Display Start","Tall Wall Display Mid","Gondola End PropBread2100Hx1464W BurntG","Refrig End Prop Bread 2100Hx1760W BurntG","Low Wall Display Start","Low Wall Display Mid","Wire Basket Long")
  , MaterialMaster_FOOD_301_400 = c("Wire Basket Round","Wall Display Standard Start/End MK4","Wall Display Standard Add MK4","Wall Display 5 Shelf Kit MK4","Wall Display Naked Kit MK4","Wall Display Naked Wire Kit MK4","Set Bakery Train Kit MK4","Set Bakery Train End MK4","Table Assembly Impulse MK4","Frame Hanging Sign MK4","Assembly Cake Box Wall Unit MK4","Base Cake Box MK4","Panel Wall Unit End Left MK4","Panel Wall Unit End Right MK4","Basket Round Wire MK4","Ticket Folded Pack of 100 MK4","Wall Unit 1200mmHx915mmW MK5 BAKERY","Wall Unit 1700mmHx915mmW MK5 BAKERY","Under Counter / Table Display MK5 BAKERY","Table Train 3 Bay MK5 BAKERY","Train Corner Unit MK5 BAKERY","Counter Naked Bread 1 Bay MK5 BAKERY","Counter Naked Bread 2 Bay MK5 BAKERY","Counter Naked Bread 4 Bay MK5 BAKERY","Table Impulse MK5 BAKERY","Wall Unit 1200mmHx450mmW MK5 BAKERY","Wall Unit 1700mmHx450mmW MK5 BAKERY","Table Train 2 Bay MK5 BAKERY","FrameSign DisplayUnderCounter MK5 BAKERY","TrayHolder Counter NakedBread MK5 BAKERY","StorageUnit CounterNakedBread MK5 BAKERY","WallUnit NakedBread Perimeter MK5 BAKERY","Sticker x 9 Bin for paper use Bakery","Sticker x 9 Grease Proof Paper Bakery","Sign Bakery Sourdough 600x1300mm","Gondola End PropBread2100Hx1464WClearMat","Refrig End Prop Bread 2100Hx1760WClearMa","Wall Display 6 Shelf Kit MK4","Bakery B/Dock Dry Rack Kit-6 Beam","Rack Cooling 12 Tray Endload S/Steel","Stand A4 PLU","Trolley S/ Steel Bread Tin Storage","Rack Cooling 18 Tray Endload S/Steel","Rack Cooling 12 Tray Endload S/Steel","Bench Wrapping S/Steel 1500x760mm Mobile","Rack Cooling S/Steel With 5 Baskets","Shelf Microwave S/S 445mm x 490mm","Bench Depanning SSteel Castor 1380x762mm","Bakery Rack 1300 Chrome","Rack for Condiment Storage Containers","Storage Containers for Condiments","Bench Pastrycook S/S 3000x750x975 Mobile","Kit Teflon Sheets Tub Rack","Kit Teflon Sheets Tub Rack \\(Moffat\\)","Fridge Coke FEM \\(Skope\\)","Fridge Coke Checkouts MetroBPS","Fridge Coke SCO MetroBPS","Gate Checkout","Autogate double double","Autogate single double","Bin Brush Finished 10 Litre","Trolley Baby Seat Belt","Cash Bag Heavy Duty 150x100mm","Cash Bag Canvas 291x229mm","Canister/Dispenser-Hygiene Station\\(LVA\\)","Stand Wet Umbrella - Automatic","KitDividers W/Pushers 1Bay 918WCig Draws","KitDividers W/Pushers 1Bay CigDraws\\(QLD\\)","KitDividers W/Pushers 1 Bay 513WCigDraws","Bracket Z for Registers","Holder with Acrylic Dummy Phone LVA","Holder Acrylic Brochure Free StandingLVA","Holder Dummy Phone Sml with bracket LVA","Holder Dummy Phone Lrg with bracket LVA","Holder Acrylic Brochure LVA","Kit Dummy PhoneHolder/Brkt/Acry BrochSML","Kit Dummy PhoneHolder/Brkt/Acry BrochLRG","Defibrillator HS Samaritan PAD 500P","Floor card Unit","Bar merchandiser - pocket size shopping","Bag merchandiser single use plastic/reus","Basket ticket pack \\(pkt of 10\\)","Baskets Hand Black 30Lt with WW Logo","Rolla Baskets Black 30 Ltr with WW Logo","Rolla Baskets Green 30 Ltr with WW Logo","Basket Hand Green 30 Ltr W/AFFP Logo","Mat Anti Fatigue Interlock Centre61x91cm","Data Strip Kit for Kiosk \\(pk 56\\)","Bracket Scanner Rewards","Stand Enviro Bag for Checkout","Bin Wheelie 240lt Red Recycled Plastic","Kit Retrofit Merch Tandem Coke","Free Standing Multibag stand","Low Height Multibag stand","Low Height Fabric bag stand","3 Tier Multibag stand","Low Profile Basket - Reusable bags","Counter Wire Tray - Reusable Bags","Counter Card Unit - Reusable bags","Floor Wire unit - Reusable bags")
  , MaterialMaster_FOOD_401_500 = c("Flipscan Hook - Express checkouts","Fabric Bag Shelf","Chiller Bag Shelf","Bin Bag Recycled","Merch GiftCard EndBay1103Lx1376W W/Hooks","Frame V Ticketing 800x695mm","SCO Mini Merchandiser Magazine Unit","Hook Kit Merch Gift Card End Bay","Merchandiser Front End Confectionery","Merch Tandem C/Out Confec CopperKinetic","Ambeint water topper OD290F","Ambeint water topper OD330F","Basket Twin Trolley Stand MK2","Tensabarrier Receiver Post with 2m Tape","Tensabarrier complete unit with A4Frame","Merchandiser Front End 3.5m Checkout","Service Desk Storage Module ShimmChamp","Service Desk Bin Module ShimmChamp","Basket Twin Trolley Stand","SCO MiniMerch Confec 2017 ShimmChamp","SCO MiniMerch Mag Unit 2017 ShimmChamp","Unit Display Frame Smoking Policy NSW","Stand Bag Holder for F/Barrels 1010mm H","Merch single co non fridge 4.2m SemiTD","Merch tandem co non fridge 4.2m SemiTD","Merch tandem co fridge 4.2m SemiTD","Merch single co non fridge 3.5m SemiTD","Merch tandem co non fridge 3.5m SemiTD","Merch tandem co fridge 3.5m SemiTD","Merch single co non fridge 3.0m SemiTD","Merch tandem co non fridge 3.0m SemiTD","Merch tandem co fridge 3.0m SemiTD","SCO Mini merch Confec SemiTD","SCO Mini merch Mags SemiTD","Merch mags & News SemiTD","Merch Confec SemiTD","Kit Low Height Wall 1200mm End","Kit Low Height Wall 2400mm Start","Kit Low Height Wall 2400mm Mid","Kit Low Height Wall 2400mm End","Kit 90 Degree Corner","Wall End Cap","Infill Stainless Steel 1200mm","Infill Stainless Steel 2400mm","Kit T Join Cap & Corner","Kit 45 Degree Corner","Merch GiftCard EndBay WHooks SemiTD","Frame V Ticketing SemiTD","StandTwin Service&Screw NCR Mon Sat Blck","Merch Single C/O NonRefrig 4.2m EShimChm","Merch Tandem C/O NonRefrig 4.2m EShimChm","Merch Tandem C/O Refrig 4.2m EShimChm","Merch Single C/O NonRefrig 3.5m EShimChm","Merch Tandem C/O NonRefrig 3.5m EShimChm","Merch Tandem C/O Refrig 3.5m EShimChm","Merch Single C/O NonRefrig 3.0m EShimChm","Merch Tandem C/O NonRefrig 3.0m EShimChm","Merch Tandem C/O Refrig 3.0m EShimChm","Merch Mag & Newspaper EShimChm","Merch Confectionery EShimChm","MerchTan Front Only C/O NonRefrig EShChm","Merch Tandem Front Only C/O Refrig EShim","Merch Tandem Overbelt C/O 4.2 EShimChm","Merch Single C/O NonRefrig 4.2m EShimChm","Merch Tandem Overbelt C/O 3.5 EShimChm","Merch Single C/O NonRefrig 3.5m EShimChm","Merch Tandem Overbelt C/O 3.0 EShimChm","Merch Single Overbelt C/O 3.0 EShimChm","Merch Mag & Newspaper EShimChm","Merch Confectionery EShimChm","Stand Britex Large - Low Profile","C/O Express Mag Conveyor MK3 LH","C/O CarouselExtTandemCopperKineticKESHUN","C/O Carousel Ext RH CopperKinetic KESHUN","C/O Carousel Ext LH CopperKinetic KESHUN","Checkout CopperKinetic\\(TS&B\\) 3.0mtr RH","Checkout Tandem \\(Keshun\\) 4.2 Metro BPS","Checkout RH \\(Keshun\\) 4.2 Metro BPS","Checkout LH \\(TS&B\\) 3.0m MetroBPS","Checkout Tandem \\(KESHUN\\) 3.0m MetroBPS","C/O ExpressMag LH 1300x1281 MetroBPS","C/O ExpressMag RH 1300x1281 MetroBPS","Pole Oval Blank MetroBPS","Pole Oval Dual Mod C/Out MetroBPS","Pole Oval Send Kiosk MetroBPS","Checkout Tandem \\(TS&B\\) 4.2m EShimChm","Checkout LH \\(TS&B\\) 4.2m EShimChm","Checkout RH \\(TS&B\\) 4.2m EShimChm","Checkout Tandem \\(Keshun\\) 4.2 EShimChm","Checkout LH \\(Keshun\\) 4.2 EShimChm","Checkout RH \\(Keshun\\) 4.2 EShimChm","Checkout Tandem\\(TS&B\\)3.5m EShimChm","Checkout LH \\(TS&B\\) 3.5m EShimChm","Checkout RH \\(TS&B\\) 3.5m EShimChm","Checkout Tandem \\(KESHUN\\) 3.5m EShimChm","Checkout LH \\(KESHUN\\) 3.5m EShimChm","Checkout RH \\(KESHUN\\) 3.5m EShimChm","Checkout Tandem \\(TS&B\\) 3.0m EShimChm","Checkout LH \\(TS&B\\) 3.0m EShimChm","Checkout RH \\(TS&B\\) 3.0m EShimChm")
  
  , MaterialMaster_FOOD_501_600 = c("Checkout Tandem \\(KESHUN\\) 3.0m EShimChm","Checkout LH \\(KESHUN\\) 3.0m EShimChm","Checkout RH \\(KESHUN\\) 3.0m EShimChm","C/O ExpressMag LH 1300x1281 EShimChm","C/O ExpressMag RH 1300x1281 EShimChm","Pole Oval Blank EShimChm","Pole Oval Dual Mod C/Out EShimChm","Pole Oval Send Kiosk EShimChm","Bench Customer Recycled Bags 1100mm","Bench Customer Recycled Bags 1800mm","Bench seat - Replas 1800 NW","Bench seat - Replas 1100 NW","Centre Unit CopperKinetic KESHUN","Express c/out bin module SemiTD","Express c/out storage module SemiTD","Pole Oval Blank SemiTD","Pole Oval Dual Mod C/Out SemiTD","Pole Oval Send Kiosk SemiTD","Board Managers 700x450x3mm","Back Panel for V Ticket 1130x695mm","Kit Sign Checkout Light Vinyl Std \\(1-6\\)","Kit Sign C/out Light Vinyl Std Sngl Num","Price Board Tobacco 340x700","Board Price Tobacco WA 342x704mm","Board Price Tobacco 340x702mm","Board Price Tobacco WA/TAS 340x734","Light C/O LED","Service Desk Quarry RH","Service Desk RH 4026X4088 Quarry","Service Desk 2 Express LH \\(DHG\\)","Service Desk 3 Express LH \\(DHG\\)","Service Desk 4 Express RH \\(DHG\\)","Service Desk 2 EXPS LH MK1 Weathered Oak","Service Desk 2 EXPS RH MK1 Weathered Oak","Service Desk 3 EXPS LH MK1 Weathered Oak","Service Desk 3 EXPS RH MK1 Weathered Oak","Service Desk 4 EXPS LH MK1 Weathered Oak","Service Desk 4 EXPS RH MK1 Weathered Oak","Service Desk 2 EXPS LH MK1 Weathered Oak","Service Desk 2 EXPS RH MK1 Weathered Oak","Service Desk 3 EXPS LH MK1 Weathered Oak","Service Desk 3 EXPS RH MK1 Weathered Oak","Service Desk 4 EXPS LH MK1 Weathered Oak","Service Desk 4 EXPS RH MK1 Weathered Oak","Buffer Floor 66x66 Stained H/Wood Finish","ScreenSecurity 750GL 1050x1200Add Sorrel","Screen Security 750GL 1050x1200 Start So","Service Desk 3 EXPS LH Ext Counter W/Oak","Service Desk 3 EXPS RH Ext Counter W/Oak","Service Desk 4 EXPS LH Tassie Oak","Service Desk 4 EXPS RH Tassie Oak","Service Desk 4 EXPS LH Tassie Oak","Service Desk 4 EXPS RH Tassie Oak","Service Desk 3 EXPS LH Tassie Oak","Service Desk 3 EXPS RH Tassie Oak","Service Desk 3 EXPS LH Tassie Oak","Service Desk 3 EXPS RH Tassie Oak","Service Desk 2 EXPS LH Tassie Oak","Service Desk 2 EXPS RH Tassie Oak","Service Desk 2 EXPS LH Tassie Oak","Service Desk 2 EXPS RH Tassie Oak","Service Desk 3 EXPS LH Tassie Oak","Service Desk 3 EXPS RH Tassie Oak","Checkout Side Walk Sale Stipple Seal","Trolley Std 100lt with TS&B Drink Handle","Trolley Std 212lt with TS&B Drink Handle","Trolley 100lt MKL Trav/W Tube/H C/L Wht","Trolley 100lt MKL Std/W Tube/H C/L Wht","Trolley 100lt QHDC Trav/W Tube/H C/L Wht","Trolley 100lt QHDC Std/W Tube/H C/L Wht","Trolley212ltMKL T/S Std/W Tube/H C/L Wht","Trolley212 MKL T/S Trav/W Tube/H C/L Wht","Trolley212 QHDC T/S Std/W Tube/H C/L Wht","Trolley212QHDC T/S Trav/W Tube/H C/L Wht","Trolley 100lt QHDC Std/W Tube/H Green","Trolley212QHDC Fam1xCap1xSeatS/W TH Grn","Trolley212 QHDC FamT/Cap Std/W TH Grn","Trolley QHDC Wheel Chair","Trolley QHDC 110 Litre Grey PUR Wheels","Trolley Kiddie Car","Lock Coin&Chain Suit PistolGrip Handles","Handle W/O Logo&Lock CafePistolTrolley","HandleWithLogo&CoinLockCaféPistolTrolley","Handle Tube 100L W/End Caps","Handle Tube 212L W/End Caps","Trolley 212lt MKL T/S Std/W Tube/H","Trolley 212lt MKL T/S Trav/W Tube/H","Trolley 212lt MKL T/S Std/W Cafe/H","Trolley 212lt MKL T/S Trav/W Cafe/H","Trolley100TravWheel W/Drink Hdle C/L MKL","Capsule Trolley Baby Red","Trolley L/Handle S/Variloc$1&2","Trolley Lock Systec Variloc MasterKey $1","Handle Comfort for MKL 212L Trolley","Handle Comfort for QHDC 212L Trolley","Trolley QHDC Convenience Shop","Trolley 212 QHDC FamT/Cap Std TH Wht","Trolley 212lt QHDC T/S Std/W Tube/H Wht","Trolley212QHDC Fam1xCap1xSeat Std TH Wht","Trolley212QHDC Fam1xCap1xSeatTrav TH Wht")
  , MaterialMaster_FOOD_601_700 = c("Trolley 212lt QHDC T/S Trav/W Tube/H Wht","Trolley 212lt QHDC FamT/Cap Trav TH Wht","Trolley 212lt QHDC FamS/CapTrav/W TH Wht","Capsule Baby For Family Trolley","Trolley 212QHDC FamS/CapStd TH Wht","Trolley100L Zinc Std W/AS Wheel NoHandle","Trolley100L Zinc Std W/A Wheel No Handle","Trolley 212L Std W/AS Wheel No Handle","Trolley212L Zinc Trav W/FtBrake NoHandle","Trolley 100lt QHDC Std/W Tube/H Wht","Trolley 100lt QHDC Trav/W Tube/H Wht","Trolley 212lt QHDC S/S Std/W Tube/H Wht","Trolley 100lt MKL Std/W Tube/H Wht","Trolley 100lt MKL Trav/W Tube/H Wht","Trolley 100lt QHDC Std/W Cafe/H","Trolley 100lt QHDC Trav/W Cafe/H","Trolley 212lt QHDC S/S Std/W Cafe/H","Trolley 100lt MKL Std/W Drink/H","Trolley 100lt MKL Trav/Wh Drink/H","Trolley 212lt MKL S/S Std/W Drink/H","Trolley Special Needs","Trolley 212lt QHDC T/S Trav/W Cafe/H","Trolley212QHDC Fam1xCap1xSeatTrav CH","Trolley 212lt QHDC FamT/Cap Trav/W CH","Trolley212 QHDC T/S Std/W Tube/H C/L Grn","Trolley 212 QHDC T/S Trav/W Tube/H Green","Trolley 212lt QHDC T/S Std/W Cafe/H","Trolley 212QHDC Fam1xCap1xSeatStd CH","Trolley212QHDCFam1xCap1xSeatTrv TH Green","Trolley 212lt QHDC Family T/CapStd/W CH","Trolley 212lt QHDC FamT/CapTrav TH Green","Trolley 212lt QHDC T/S Std/W Tube/H Wht","Trolley 212lt QHDC T/S Trav/W Tube/H Wht","Trolley212QHDC Fam1xCap1xSeat Std TH Wht","Trolley212QHDC Fam1xCap1xSeatTrav TH Wht","Trolley 212 QHDC FamT/Cap Std TH Wht","Trolley 212lt QHDC FamT/Cap Trav TH Wht","Trolley 100lt QHDC Trav/W Tube/H Wht","Trolley 212lt MKL T/S Std/W Tube/H","Trolley 100lt MKL Trav/W Tube/H Wht","Trolley 212lt MKL T/S Trav/W Tube/H","Stand Hygiene Unit 1564x400","Grill Panini Pyrocream FAMA","Blender Vitamix Quietone","Jug 1.4Lt for Vitamix","Bin75L RC262000Gray RC26400BLA","Basket Wicker","Bottle Squeeze 2 Lt","Container Air Tight 1 Lt","Knife Bread","Dome Cake","Stand Cake 70mm High","Spatula Flexible Nylon Handle","Stand Ticket Pack of 12","Tongs for Mashmellows 175mm","Tongs Pastry","Tongs Sandwich Flat","Brush","Sugar Shaker","Teaspoon","Tablespoon","Milk Jug","Bucket","Red Cloths \\(Chux\\)","Green Cloths \\(Chux\\)","Board Display Mid Shelf 350x175x20mm","Docket Book No8 Tri 170x100mm 50sht","Spike Docket Chrome","Holder Docket S/S 750mm","Jar A/H Montana Glass/Black Lid 2.9","Chair Galvanised with Timber Seat \\(Café\\)","Chair High Café","Table with adjustable feet Cafe","Scale Customer Self Service","Fresh cut pushers L/Profile 4 bay","Fresh cut pushers L/Profile 5 bay","Fresh cut pushers L/Profile 6 bay","Clip Large with Large Card Holder","Knife Swibo Produce 250mm","Knife Swibo Produce 310mm","Clip Spring Clear Plastic 370mm Stem","Holder Instructions CPN PETG Pk 60","Cabinet Cloth 1000mm x 18 Metre Roll","Shelf Talker Plastic 105 x 99mm PK25","Shelf Talker Plastic Mushrooms105x99Pk25","Mat Safety 914x1524mm Green Scraper","BinWheelie80LtrLimeGreenLidFoodRecycling","Kit Brush Produce","Cloth Cab Black 1000mm x 18 Metre Roll","Post for Plastic Bag Slope Table&Nut Dis","Post for Plastic Bag Cabinet Universal","Post for Plastic Bag Metal Tables Black","PostforPlasticBagSlopeTable&PotatoeBlack","Post forPlasticBag CabinetUniversalBlack","Basket Rattan Plstic295x380x120 Brwn 22B","Basket Rattan Plstic575x385x120 Brwn 22B","Basket Rattan Plstic575x385x175Brwn 22B","Basket Rattan Plstic780x200x120Brwn 22B","Basket Rattan Plstic885x200x120Brwn 22B","Holder Ticket - Produce Table L/H")
  , MaterialMaster_FOOD_701_800 = c("Holder Ticket Produce Slope Bin","Holder Ticket - Produce Table R/H Black","Holder Ticket - Produce Table L/H Black","Holder Ticket Produce Table Centre Black","DividerSuitDisplayTableMetal750x166Black","Kit Dividers 1Bay Bag Fresh Cut Imperial","Kit Dividers 2 Bays Bag Fresh Cuts","Kit Dividers 3 Bays Bag Fresh Cuts","Kit Dividers2 Bays Bag Fresh Cuts Metric","Kit Dividers3 Bays Bag Fresh Cuts Metric","Insert Produce 5 trenches 600x790mm","Insert Produce 6 trenches 600x790mm","Insert Produce 7 trenches 600x790mm","Tray Vac Formed Fruit Separator Small","Tray Vac Formed Fruit Separator Large","Tray Vac Formed Fruit Separator Medium","Tray Vac Formed Fruit Separator Black","Display Nut D/S Unit W/Bag Holder 1 Bay","Floral Unit 3 Tier RH Black","Floral Unit 3 Tier LH Black","Steps 3 Tier Produce Black","Floral Unit 3 Tier Mid Blk W/Gal Buckets","Floral Module - Corner Unit MK4","Floral Module - Straight MK4","Floral Module - Internal Corner MK4","Floral Module-Corner Unit T/Oak MK5","Floral Module-Straight T/Oak MK5","Floral Module-Internal Corner T/Oak MK5","Holder Floral Display Bag","Frame Gantry Produce 1800mm SemiTD MK5","Frame Gantry Produce 900mm SemiTD MK5","Frame Gantry Promo Bin SemiTD MK5","Frame Gantry Sendai Fridge SemiTD MK5","Scales Produce Gantry SemiTD MK5","Bin Produce 800x1200 Promo T/Oak MK5","Table Produce Cnr Protector SemiTD MK5","Kit Sendai Cladding Fix","Scale Pole Produce - Black","Table Produce1800mm w/Spillover T/OakMK5","Table Produce900mm w/Spillover T/Oak MK5","Table ProduceRH End w/Spillover T/OakMK5","Table ProduceLH End w/Spillover T/OakMK5","Lettuce Wall 1/2 Imperial","Lettuce Wall 1/2 Metric","TableProduce1800mmw/SpilloverS/Blk W/Oak","TableProduce 900mmw/SpilloverS/Blk W/Oak","Table Produce RHEnd SpilloverS/Blk W/Oak","Table Produce LHEnd SpilloverS/Blk W/Oak","Display Avocado inc Signage Gantry W/Oak","Table Produce 1800mm T/Oak MK5","Table Produce 900mm T/Oak MK5","Table Produce End LH Door T/Oak MK5","Table Produce End RH Door T/Oak MK5","Table Produce BananaLH Door T/Oak MK5","Table Produce BananaRH Door T/Oak MK5","Table Produce LH End Panel T/Oak MK5","Table Produce RH End Panel T/Oak MK5","2nd Tier Produce Shelf 1800mm SemiTD MK5","2nd Tier Produce Shelf 900mm SemiTD MK5","Bin Produce Promo T/Oak MK5","Scale Arm Produce SemiTD MK5","Divider Produce Table SemiTD MK5","Holder Produce Bag SemiTD MK5","Produce Table Signage Frame 1600","Produce Table Signage Frame A3","Stand Flower Display Large 682x436mm","Stand Flower Display Small 341x436mm","Nut Island Unit Weathered Oak","Shelf 410mm Imperial 15-30DEG","Shelf 410mm Imperial 0-15DEG","Shelf 510mm Imperial 0-15DEG","Product Fence Wire 75mm Imperial Black","Shelf 410mm Metric 15-30DEG","Shelf 410mm Metric 0-15DEG","Shelf 510mm Metric 0-15DEG","Product Fence Wire 75mm Metric Black","Kit ABS Shelf/Metr 1 Bay Fancy Lettuce","Kit ABS Shelf/Metr 1 Bay Leafy Veg","Kit Acryl Shelf/Metr 1 Bay Fresh Herbs","Kit ABS Shelf/Metr 1 Bay Cut Vegetables","Kit ABS Shelf/Metr 1 Bay CutFruit NoMirr","Kit Bowls Assorted Veg 1 Bay Bowls Imp","Fresh cut pushers 2 bays","Fresh cut pushers 3 bays","Fresh cut pushers 4 bays","Fresh cut pushers 1 bays","Fresh cut pushers 5 bays","Fresh cut pushers 6 bays","TableDisplayMetalW/FlatShelf1200x790Blac","PanelEndDisplayTable Metal759x351RHBlack","PanelEndDisplayTableMetal759x351LHBlack","Banana Half Round End Black","Kit Banana Display Complete Black","Stand Free Fruit for Kids","Table single sided \\(Local sourced\\)","Sendai 2 Berry Cladding w/Gantry MK5","Sendai 2 Berry Cladding w/Gantry MK4","Kit ABS Shelf/Imp 1 Bay Leafy Veg","Kit Acryl Shelf/Imp 1 Bay Fresh Herbs","Kit ABS Shelf/Imp 1 Bay Cut Vegetables")
  , MaterialMaster_FOOD_801_900 = c("Kit Acryl Shelf/Imp 1 Bay Mushrm NoMirr","Kit ABS Shelf/Imp 1 Bay Organics","Kit ABS Shelf/Imp 1 Bay CutFruit NoMirr","Lettuce Wall Imperial","Lettuce Wall Metric","Display Avocado w/Sign Gantry T/Oak MK5","Nut Island Unit 510 Shelf Tas Oak MK5","Nut Island Unit 510 Shelf Weathered Oak","KitABS ShelfMetric 1Bay asparagas/brocco","Kit ABS Shelf Metric 1 Bay cut berries","Kit ABS Shelf Metric 1 Bay Cut Vege","Kit ABS Shelf Metric 1 Herbs","Kit ABS Shelf Metric 1 Bay Iceberg lettu","Kit ABS Shelf Metric 1 Bay leafy vege","Kit ABS Shelf Metric 1 Bay mushrooms","Kit ABS Shelf Metric 1 Bay Organics","KitABS ShelfImperial 1Bay asparagas/broc","Kit ABS Shelf Imperial 1 Bay cut berries","Kit ABS Shelf Imperial 1 Bay Cut Vege","Kit ABS Shelf Imperial 1 Herbs","Kit ABS Shelf Imperial 1 Bay Iceberg let","Kit ABS Shelf Imperial 1 Bay leafy vege","Kit ABS Shelf Imperial 1 Bay mushrooms","Kit ABS Shelf Imperial 1 Bay Organics","Bin Spill Over 1800mm","Bin Spill Over 900mm","Bin Produce End Table & Banana End","Holder Bag Mushroom","Table Modular with pull out","Shelf Second Tier Display","Table End Modular with LH Door","Table End modular with RH Door","Kit Banana Modular","Table metal divider","Set Wooden Box","Bin Promo","Panel Produce Table LH End","Panel Produce Table RH End","Holder Floral Display Bag","TableProduce 1800mmMK4 SatinBlk/W Oak","Table Produce 900mm MK4 SatinBlk/W Oak","Table Produce EndLH DoorMK4SatinBlk/WOak","TableProduce EndRH DoorMK4SatinBlk/W Oak","Table Banana LH Door MK4 SatinBlk/W Oak","Table Banana RH Door MK4 SatinBlk/W Oak","TableProduceLH EndPanelMK4SatinBlk/W Oak","TableProduce RHEndPanel MK4SatinBlk/WOak","TableProduce 2ndTierShelf 1800mmMK4S/Blk","TableProduce 2nd TierShelf 900mmMK4S/Blk","Bin Promo MK4 Satin Blk/W Oak","Table Produce Banana Kit","Table End Scale Arm MK4 Satin Black","TableProduce SingageFrameA3 MK4Satin Blk","Holder Bag MK4 Satin Black","Table Metal Divider MK4 Satin Black","Set Wooden Box MK4","Slanted Display set Magnet PK6 MK4","Shelf Metric B/Satin 1246mm w/brktKM20-4","Sign Ticketing A6 Flower","Sendai 2 Berry Fridge Cladding w/gantry","Bin Potato Timber 1200x1162 Black","Panel End Potato Bin 918x1012 LH Black","Panel End Potato Bin 918x1012 RH Black","Trolley F&V Prep S/Steel with Stop Bar","Board Cutting White 585x400x15 for F12","Trolley Stock for 2 Crates 865x600","Fogging System Generator Kit","Fogging System Diffusion Manifold","Fogging System Generator Shroud","Freezer Double Door","Cleaner Wet'n'Dry Vacuum 1000watt","Calculator Heavy Duty Print","Counter Coin Mach 3","Counter Auto Note","Printer + Cable Note Counter","Training on Equipment for Staff","Washer Pressure Karcher","Cleaner Steam & Wet/Vac 5 Litre","Trolley Zinc for Platform Scale","Scale Digital Platform for Back Dock","Baler Combo Cardboard/Plastic Auto","Baler Bramidan X25","Baler Bramidan B5W","Fax Brother BRMFC-L2730DW","Liners Table Baby Change Rubbermaid","Stand Acrylic Deli Platter Brochure Hold","Bullion Bag 406 \\(Freight Included\\)","Kit Cleaning Equipment Standard","Kit Spill Supermarkets Complete w/Video","Kit Brush BOH","Bin 30Ltr RedLidGeneralWasteL/fillOffice","Bin 30Ltr BlueLidCardboard RecycleOffice","BinWheelie120OrangeLidSoftPlasticR/cycle","Display System Wall 10 \\(Durable\\)","Grip Super POS Pk100","Bin White 45L Step on Pedal","Bin Blue 120L w/lifta Lid","Whiteboard 1200mmx900mm","Pinboard 1200mmx900mm","Whitebrd Pinbrd KIT Lge 3xWB 2xPB 1xProm")
  , MaterialMaster_FOOD_901_1000.1 = c("Whitebrd Pinbrd KIT Sml 3xWB 2xPB 1xProm","Tape Packaging GREEN PRODUCE","Tape Packaging RED MEAT","Tape Packaging BLUE SEAFOOD","Tape Packaging YELLOW BAKERY","Tape Packaging BROWN DELI","Tape Packaging PINK FRONT END","Tape Packaging GREY GROCERY","Tape Packaging BLACK BACK OF HOUSE","Tape Packaging ORANGE LIQUOR","Tape Pack WHITE ON GREEN GENERAL","Mirror Locker Room 1800mm x 600mm","Mirror Accessible Toilet 950mm x 600mm","Kit Start Twine Baler Bramidan")
  , MaterialMaster_FOOD_901_1000.2 = c("Dolly for 75L Grey Container EQCF2001","Glass Highball \\(Pack of 24\\)","Platter Ryner Dish White 500x200x70mm","Platter Flat 395x270mm","Gizmo Ceiling Twist-On Grid Clip","Pole Telescopic for Gizmo","Shelf Storage Locker Room Putty","Bowl Fruit - Team","Stool Black Vinyl for Work Station Fog","Desk Branch Managers with RH Return","Chair Ergo Typist Grey 3 Wheel","Cradle RF Gun for Back Dock Workstation","Chair Office Milano Rio Black","Chair Lunchroom Padded","Table Planning Room 3500mmx1200mm","Table Planning Room Small 1800x1200mm","Table Dining Lunchroom 1500mmx800mm","Desk Branch Trainer with Lockable Drawer","TV 55inch TCL","Cabinet filing two drawer putty","Shelf hangingw/coat rail forlocker putty","Cupboard Sec 3Door 4 Adj shlv Mgr Office","Bench Seat Locker 1200","Desk Branch Managers with LH Return","Lounge 2 Seater Home","Bolet Pendant","Locker 3 Door - FRESH SPRING","Coffee Table","Table Dining Rectangular 1500x800mm","Single Cubby Pitched Roof","Stool French Industrial Kitchen Black","Table Coffee Zinc 1180mmx650mm","Chair Industrial Emperor - Lounge Chair","Table Coffee Lunchroom Supplementary","Table Coffee Vernier","Signage Safety External BackDock 900x600","Board Emergency Evacuation 990x660","Sales Promotion Board 1800mm x 1200mm","Stand \"PleaseTakeOne\"A4 StoreMap MetStrm","Sign for \"Please Take One\" Stand 297x210","Kit Sign Free Standing Disruption","KitSignBakeryHoardDisrupt\\(Mth.EPMtoAdvis","KitSignB/cherHoardDisrupt\\(Mth.EPMtoAdvis","KitSignCafeHoardDisrupt\\(Mth.EPMtoAdvise\\)","KitSignDeliHoardDisrupt\\(Mth.EPMtoAdvise\\)","KitSignP/duceHoardDisrupt\\(Mth.EPMtoAdvis","KitSignSeaF.HoardDisrupt\\(Mth.EPMtoAdvise","KitSignSushiHoardDisrupt\\(Mth.EPMtoAdvise","Kit Sign Months Disruption","Sign Disruption 12 Month kit 210mmx55mm","Sign External Fence Shadecloth DP","Kit Sign Line 8 Instore Hoarding DP","Tote for Online Stores","Platform Milk","Frame Pallet 2/3 CT \\(2/3 Pallet Frame\\)","Safe 2 Key 718x668x663","Safe Europa 634 Lt 1796x903x773mm size 7","Safe Tank Model CS1 \\(two key facility\\)","Trolley Parcel Pickup 1560x610x1650 Ch","Trolley Online \\(6Totes\\) W/Plastic Sleeve","Pallet Racking 2750x2400x840 Start Bay","Pallet Racking 2750x2400x840 Add Bay","Pallet Rack 1375x2400x840 Start 1/2 Bay","Pallet Racking 1375x2400x840 Add 1/2Bay","Safe Referb 2 Key Small","Safe Referb Large","Forklift Crown LPG CG20SC NT Stores ONLY","Tool Box Medium Lift Out Tray","Step Safety Step Up Refurbishment Kit","Pallet Mover With Battery","Kit 1 Bay Equip Reserve 2100x1850x610mm","Kit 1 Bay Damaged Stock 2100x1850x610mm","Trolley Stock Replenishment 2 Deck","Storage Unit Mobile for POS Posters","Lifter Bin Wrightway for Flat Dock","Lifter Bin Wrightway for Recessed Dock","Box Security for Infra-Red Temp Gun","Mailbox Heavy Duty Galvanised","Table Change Baby Rubbermaid","Bandsaw Barnes LRG","Bandsaw Barnes JNR","Refurbed Biro Mincer","Mat Meat Comfort Flow Black 600mmx900mm","Mat Meat Comfort Flow Black 850mmx1450mm","Tub Nally 16 Kg Red","Lid White for M41 No 7 Nally Tub")
  
  , MaterialMaster_FOOD_1001_1100 = c("Stone Sharpening Fine/Coarse 108","Kit Brush Meat","Tray Polycarbonate Blck 460x350 Carlisle","Tray Styrene Black 406x294x42mm","Board Demonstration Chopping Red","Bench Scraper Red","Tray Meat 410 x 156 x 24 mm","Tray Meat 418 x 161 x 56 mm","Tray Meat 204 x 25 mm","Tray Meat 314 x 208 x 58 mm","Tray Meat 360 x 310 x 25 mm","GraphicWall ButcherPrimary3000x1300 100%","Holder Information Card 2 Pocket","Bench Pack 1200x620x800mm w/out base M2","Bench Packing 1200x600x800mm","Bench Packing 1800x600x800mm","Bench Boning 1800x600x800mm M6","Bench Boning 1200x600x800mm M7","BenchCuttingW/BlueBoardsForDry Aged Beef","Trolley Galvanised For Fat & Bone Tubs","Trolley Nally Tub for Meat Mincer","Conveyor Roller S/Steel 1800mm \\(6'\\)Start","Conveyor Roller S/Steel 1800mm \\(6'\\)Add","Board Cutting White 600x450x15mm for M6","Board Cutting White 600x400x15mm for M7","Weight Test 10kg for Scales","Kit Brush Dairy Coolroom","Kit 1 Shelf Optimal S/Goods & Sh/Cheese","Kit 1 Shelf Optimal Entertain/Rollmop","Kit 1 Shelf S/Goods & Sh/Cheese Metric","Kit 1 Shelf Entertain/Rollmop Metric","Riser Front 40mm w/mag tape SGood/Cheese","Rear Piece for riser w/mag tape SGd/Chse","D/Strip Clip On Frozen Food 148mm Black","D/Strip Glass Door Provista 750mm Black","Data Strip Clip On Provista 1220mm Black","D/Strip Glass Door Provista 760mm Black","Data Strip Clip On Provista 1245mm Black","D/Strip ClipOn Frozen Food 148mm S/White","D/Strip GlassDoor Provista 760mm S/White","DataStrip ClipOn Provista 1245mm S/White","Holder Ticket A5 Coffin Freezer","Holder Ticket - Chest Freezer Satin Blac","DataStripAngled 39mm Black 1220mm FB IMP","Data Strip Angled 39mm Black 1245mm F/B","Data Strip Angled 39mm Black 750mm F/B","Data Strip Angled 39mm Black 760mm F/B","Data Strip Angled 39mm Wht 1245mm F/B","Data Strip Angled 39mm White 760mm F/B","Data Strip Angled 39mm White 914mm F/B","Data Strip Hinged 39mm Black 1220mm F/B","Data Strip Hinged 39mm Black 1245mm F/B","Data Strip Hinged 39mm White 1245mm F/B","Data Strip Hinged 39mm White 914mm F/B","Data Strip Hinged 39mm White 914mm","Data Strip Mid Shelf 912.5 White 40mm","Data Strip Flex Roller Mid Shelf White","Shelf mat lge 2 Row D600xW600xH81 pkt 20","Shelf mat med 4 Row D600xW300xH81 pkt 20","Basket Base Dairy Metric Black 1245x575","Basket Base 80mm Front 1220 x 574 Blk","Divider Base 80mm Front 590x144 Blk","Basket Base 80mm Front 1220 x 574 White","Basket Base Dairy Metric 1245x575 White","Divider Base 80mm Front 590x144 White","Shelf with Brackets Kysor Dairy Cabinet","Basket with Brackets Kysor Dairy Cabinet","Divider for Basket Kysor Dairy Cabinet","Shelf 732mm with Brackets Kysor DairyCab","Divider Ribbed T 825 Gravity Feed System","Divider Ribbed End L825 Gravity Feed Sys","Rack Milk Flow Assembled 1 Door Kit","Shelf Milk Flow Assembled","Divider Frozen Food Well \\(ea\\)","Divider Frozen Food Well Narrow \\(ea\\)","Installation Milk Racking p/door \\(Metro\\)","Installation Milk Racking p/door\\(Regnal\\)","Divider Frozen Food Well White","Shelf with Brackets Kysor Glass Door Cab","Basket w/Brackets Kysor Glass Door Cab","Divider for Basket Kysor Glass Door Cab","Basket Base Kysor Glass Door Cab","Divider for Base Bskt Kysor GlassDoorCab","Divider Base Basket KysorGlssDoorCabXG20","Basket Base Kysor Glass Door Cab XG20","Basket Base Glass Door Freezer KG20","Shelf w/Brackets Glass Door Freezer KG20","Shelf 560mm Glass Door Freezer KG20","Shelf 1250mmDairyCaseSize 1250,2500,3750","Shelf 791mm Dairy Case Size 1590 only","Shelf 933mm Dairy Case Size 3125 only","Shelf 791mm Size 1590 Only White","Shelf933mm W/Brackets Size3125Only White","Shelf 560mm GDF White","Basket Base W/Ticket Strip White","Divider for Basket Kysor Glass Door Whit","Shelf W/Brackets GDF White","Divider for Base Basket GDF White","Bracket Universal Shelf KM20-4 Dairy","Shelf Assy.560 W/Ticket Strip&BracketsWh")
  , MaterialMaster_FOOD_1101_1200 = c("Basket Shelf Assy W/TickStrip&Bracket WH","Shelf Assy.933mmW/BracketsSignalWhKM20-4","Shelf Assy.1246 W/BracketsSignalWhKM20-4","Shelf 560mm Frigrite Glass Door Cabinet","Holder Ticket Frozen Food Well","Holder Ticket Frozen Food Well White","Milk Crate Trolley - VIC Only","Milk Crate Trolley-All States Except VIC","Dishwasher Undercounter","Wrapping Machine Coppens","Kit Film & Tray Coppens","Training for EQSD1004 Coppens Wrapper","Filter Package Water Softener Calclear","Oven Rational ChickenCooker 50A 102 Whit","Oven Convotherm C4 20 Tray","Oven Convotherm C4 10 Tray","Grease Shield Fat Separation Unit","Training for Staff on GLM300 Slicer","Slicer Deli IKSA Mankoni","TrainingStaff onIKSA DeliSlicerEQSD1126","Freight for IKSA Deli Slicer","Slicer Semi Automatic Bizerba","Slicer Auto with Rounding Plate","Blast Chiller & Stand","Hood Convo Vent +4","R/runner TwinC'mber 30/30 Oven w/fat","R/runner TwinC'mber 24/30 Oven w/fat","R/runner Single C'mber 30 Oven w/fat","R/runnerSingleC'mber24 Ovenw/fat","R/runner Single C'mber 30 Oven no/fat","R/runner Single C'mber 24 Oven no/fat","Roastrunner Fat Seperator","Hood Exhaust Twin Chamber Eyebrow","Hood Exhaust Single Chamber Eyebrow","Stand Clear 3mm Acrylic for Cane Basket","Bowl Olaria White 2.5L","Bowl Olaria Terracotta 2.5L","Shears Poultry Victorinox 7.6345 S/Steel","Knife Swibo Cooks 210mm","Glove Chain Mesh Wrist Size M Reversible","Ticket Holder Deli \\(Swivel\\)","Ticket Holder Clear 100mm With Base","Plate 305mm Diameter White T3215","Pedestal Deli Black 170x170x100mm","Step Acrylic for Deli Inline Case","BinWheelie120LLimeGrnLidF/RecyleW/L.Lift","BinWheelie80RedLidG/WasteL/fillW/L.Lifta","Cutter Cheese Wire","Kit Replacement wire \\(10\\)","Knife Scoring","Knife Double handled cheese 430mm","Spoon S/S Plastic Handle Slotted 330mm","Stand Ticket White Pk50","Stand Ticket Black Pk50","Bowl Rect Straight Angle Wht160x240x90mm","Bowl StraightSidedMel 3.0LBlk 160x240x90","Plate Square 250mm Black 250x250","Glove Steam Suregrip 20inch Med","Glove Steam Suregrip 20inch Large","Deli Step Wire 720mm","1/2 Straight Sided Melamine White","3/4 Straight Sided Melamine White","3/4 Straight Sided Melamine Black","1/2 Straight Sided Melamine Black","Bowl Triangle Full Melamine White","Bowl Triangle 3/4 Melamine White","Bowl Triangle 1/4 Melamine White","Steps Deli Bamboo set \\(3\\)","Kit Brush Deli","Ticket stand low","Ticket stand high","Riser Schnitzel Step 170mm","Riser Schnitzel Step 300mm","Riser Angled Hot Food 120mm","Steps Hot Food 2","Steps Hot Food 3","Chalkbaord 415x475x755","Crate vintage 560x370x300","Blackboard on rope hanger 400x300","Timber wire reels 600x750","Hessan roll 500mm x 10m","Crate vintage 480x210x150","Bakset 700mmH Set of 3","Bread tray retangular 500x390x150","Hamper basket square 350x350x130","Cake stand 330x 175","Black Metal Stand for Cross Merch basket","Cross Merchandise baskets","3tier DisplayTopCase Twist/fold prty pla","Artificial Wheat Grass","Galvanized Pot for Wheat Grass","Hanging Chopping Boards 480x200mm","Hanging Chopping Board 430x250mm","Basket Round Polywicker","Dish Baking 610x457x38mm","Pan Baking 318x216x38mm","Pan Baking 368x267x70mm","Stand Ambient Salami 532x1324mm","Kit Deli Bowl Range \\(Premium\\)","Stand Ticket Deli Express")
  , MaterialMaster_FOOD_1201_1300 = c("Kit Salami Rings Circle White","Stand Specialty/Local Sourcing","Shelf S/Steel Ticket Printer Wedderburn","Stand Cheese S/Format","Stand Ticket Small 50mm S/Format","Display Deli Fresh Salad Boards","Trolley&Cassette Convo Chick Cook 10:20","Trolley&Cassette Rational ChickCook10:20","Trolley,Cassette&Stand Chicken Oven10.10","Trolley 3 Tier Stainless Steel","Scaler Fish Stainless Steel","Scoop Clear P/Carb 360x143x108mm 1.9Ltr","Knife Swibo Filleting 20cm","Board Cutting","Kit Brush Seafood","Tub White Nally No 7 IHO60","Tub Blue Nally No 7 IHO60","Tub Grey Nally IHO16","Kit Ticketing \"Country of Origin\"Seafood","Trolley 12 Tier Flat Stainless Steel"
                                      ,"FriJado Multi Deck Warmer Solid Back","Juicer Citrus - Master","Juicer Ruby 2000 MkII Nutrifaster","Dispenser Juice Artic Delux - 2 Bowl","Scrubber Floor","Soup Kettle 6lt","Dishwasher Under Counter Economy - XU","Baler Vertical B3","Oven Salva Convention KX5 Single","Oven Salva Convention KX9 Single","Oven Salva Convention KX5+KX5 Stacked","Oven Salva Convention KX5+KX9 Stacked","Hood SalvaIES Extract singleduct ext o/s","Hood Salva Self condensing hood","Oven Atoll Speed","Kit Salva Tray & Rack","Shelf Tray Rack to suit 600x400 trays","Baking Perforated Tray Kit 20","Baking MatTeflonRED PK30 600x400 trays","Baking MatTeflonGREEN PK30 600x400 tray","Rack cover to suit 16 shelf tray rack","Atoll Teflon Sheets Pk10","Grill Panini Roband","PGS10 PK10 Sheets Panini Gripp","RC6 Clip Panini Grill","Cake Case Cossiga BTG RF12","Scrubber Tennant Floor Cleaner","Fridge undercounter 1DR - Glass \\(BLK\\)","Fridge undercounter 2DR - Glass \\(BLK\\)","Freezer undercounter 1DR - Solid \\(S/S\\)","Insert Banana for F&V Unit","Data Strip Clear With Hanging Tabs 100mm","Dispenser Tork Folded W4 Red/Black","Dispenser Tork Centre Feed Roll","Hook 150mm for SCO & InQueue System","Bowl Rail 365mmWx170mmD","Produce Timber Crates D/S Wall","Shelf Acrylic Clear x1 Sushi","A4 Wallmount Brochure Holder with Clips","Hook Electronic Security","Platter Rect Melamine Ryner Taroko 325x1","Platter Rect Melamine Ryner Taroko 325x2","Platter Rect Melamine Ryner Taroko 525x1","Platter Rect Melamine Ryner Taroko 530x3","Platter Rect Melamine Ryner Taroko 375x2","Platter Rect Melamine RynerTaroko368x368","Platter Melamine Ryner Taroko 330mm","Platter Melamine Ryner Taroko 430mm","Riser Gourmet Athena Art De S/S 180x180x","Riser Gourmet Athena Art De S/S 180x180x","Riser Gourmet Athena Art De S/S 180x180x","Cover Dome Round Zicco 40cm","Pan Muffin Non Stick 12 Cup","Fork Table S/S York Pk12","Knife Table S/S York Pk12","Teaspoon Table S/S York Pk12","Spoon Coffee York 18/10 Pk12","Bag Canvas Moda Oval Grey/Cream 270x160x","Bag Canvas Moda Oval Grey/Cream 250x180x","Basket Fruit Chrome Deluxe","Spoon Soda York Pk12","Cup Measuring Mini Avanti","Guard Sandwich S/S","Club Long-Turner Non Stick","Spreader Butter S/S w/Black Handle","Divider Cake Plastic Serve 8/12","Server Cake S/S Como","Mould Dariol S/S \\(Sugar Holder\\) 85x85mm","Jar Glass w/Stainless Steel Lid 5.7L","Holder Card 35mm","Cover Dome Cake Acrylic 300mm","Stand Number Table 300mm","Mop Head 350gr","Mop Bucket 16 Litre White","Mop Head Duraclean Black Socket 350gr","Mop Bucket Black 16ltr","Pastry Brush 102mm","Box Storage Cambro 305x460x150 CLR","Lid Crate Nat","Nally Crate 7 White")
  , MaterialMaster_FOOD_1301_1400 = c("Lid Cambro Storage Box 305x460x150 CLR","Nally Crate No 15","Scissors MKS take apart","Holder Docket S/S 1100mm","Dispeners Straw Auto Std R W Beckley E","Mop Handle White","Mop Handle Black","Knife Serated 30cm","Ladel Soup Small 30ml","TrayMoulded Deli NO Lip 230x300x60mm Wht","Standing Ticket- Anti-glare","Clipstrip 9 Pocket Merchandising Metal","Step Deli 610x720x160mm Black","Tub Deli NO Lip 115x230x70mm Black","Tub Deli NO Lip 115x230x70mm White","Tub Deli ABS 115x230x45mm Black","Tub Deli ABS 115x230x45mm White","Tub Deli NO Lip 230x230x70mm Black","Tub Deli NO Lip 230x230x70mm White","TrayMoulded Deli NO Lip 230x300x60mm Blk","TrayMoulded Deli NO Lip 230x150x60mm Blk","TrayMoulded Deli NO Lip 230x600x40mm Blk","TrayMoulded Deli NO Lip 300x600x40mm Blk","TrayMoulded Deli NO Lip 200x720x40mm Blk","TrayMoulded Deli NO Lip 300x720x40mm Blk","Tray Riser SG 720x200x160mm Black","Plate Rectangle Deli 300x240mm Black","Plate Rectangle Deli 300x240mm White","Plate Round Deli 240mm Diameter Black","Plate Round Deli 240mm Diameter White","Kit Fresh Food 1 Shelf","Kit Fresh Produce 1 Shelf","Flip Ticket Holder for Hooks Clear","Shelf Security W/Electronic 600mm","Shelf Security without Basic 600mm","Shelf Security W/Electronic 914mm","Shelf Security without Basic 914mm","Bin Universal 75Litre","Bracket 4 Notch 355mm S/Black","Entry Gate - Service area incl post","Table Impulse 600X600 W/Acrylic Front","Post T 1640x430x430mm L/Leg Satin Black","Post L 1640x430x430mm L/Leg Satin Black","Panel Punch Metal 902x126mm Satin Black","In Q Queuing System 2 Bay","In Q Queuing System 3 Bay","Panel Punch Metal 600x126mm Satin Black","Panel Punch Metal 600x450mm Satin Black","Cover Top Cap 600mm Satin Black","Shelf Base 600x382mmW/Buffer","EndFeatureW/SidePanels1640 EndBayS/Black","Stand Newspaper 3 Tier","BasketChiller W/HeightAdj.Shelf1134x211","Unit Bakery Tier Shelving Start Bay","Unit Bakery Tier Shelving Add Bay","Unit Produce - Feature End","Unit Produce - Centre","Panel Plain Metal 590x450mm Satin Black","Basket 598x330mm Universal NarrowFixture","MerchandiseImpulseUnit QueueKit StartBay","Merchandise ImpulseUnit QueueKitAddBay","Merchandise ImpulseUnit SCO Kit StartBay","Merchandise ImpulseUnit SCOKit Add Bay","ShelfFlatMetal In-QforSlatwall400x250x25","ShelfSlopeMetalIn-QforSlatwall400x250x25","Rack Magazine In-Q for Slatwall","Post Basket In-Q 315x180x335mm","Post Bowl In-Q 365 Diameterx170mm Deep","BowlAngledW/SlopeBracket365Diax170mmDeep","Bracket Post In-Q POS Holder \\(A4\\)","Wall Unit Produce - Start Bay With LEDS","Wall Unit Produce - Add Bay With LEDS","Acrylic Display Box W/Pers lid/Magn Catc","Panel slatwall 1900x1200","Panel slatwall 1900x600","Panel slatwall 1200Hx1200W incl trims","Panel slatwall 1200Hx600W incl trims","Panel slatwall 1600Hx1200W incl trims","Panel slatwall 1600Hx600W incl trims","Column slatwall kit gondola infill 600mm","Column slatwall kit gondola infill 915mm","Display f/stand s/wall unit 1500x400x250","Panel end trim suit register","Panel end trim suit packer","Buffer and Kick Set \\(small store\\)","Bowl Post - 5 Stacked","Kit Feature End 930x1975mm","Kit F/End W/Side Panels 930x1975mm","KitCollapsible F/End W/SidePanel930x1975","Pelmet 915mm Start Bay Macro","Pelmet 915mm Add Bay Macro","Pelmet 600mm Start Bay Macro","Pelmet 600mm Add Bay Macro","Holder Cup & Lid 3 Column","Organiser Condiment 9 Compartments","Acrylic Frid TopZ Display1000x300x100x50","Holder POS Ext InQue","Bowl Rail InQue","Drawer Cigarette Tall with Hangsell","Leg extension 100mm - Satin black")
  , MaterialMaster_FOOD_1401_1500 = c("Flower display3tier corner ext 750mm","Flower display 3tier straight 1000mm","End Bay w LEDs Feature Metro Produce","Mid Bay w LEDs Metro Produce","Gantry Sign Island Metro Produce","Impulse Table 600x600 Metro","Stand Hanging Scale Produce SS","Kit Corner Shelf x4","RegisterExpress Unit LH/RH Slatwall","RegisterExpressUnit RH W/O BagPack Unit","Stool Tolix 45cm Silver","Table Wooden 80x80x75 W/Silver Legs","Table Bar Wooden 60x60x107 W/Silver Legs","Stool Tolix 76cm Silver","Holder Bag Wall Mount","C/O ExpMagConveyor LH 1547X1461 EShimChm","C/O ExpMagConveyor RH 1547X1461 EShimChm","Kit Street Sign Holder Satin Black","Frame Chalkboard/Magnet A1 Freestanding","Flags - Now Open 300x70cm","Holder Café Menu DL","Sign Bulkhead Letters Chilled 250mm","Sign Bulkhead Letters Cafe 250mm","Sign Bulkhead Letters Bakery 250mm","Sign Bulkhead Letters Deli 250mm","Sign Bulkhead Letters Meat 250mm","Sign Bulkhead Letters Dairy 250mm","Sign Bulkhead Letters Milk 250mm","Sign Bulkhead Letters Frozen 250mm","Sign Bulkhead Letters Fruit&Veg 250mm","Sign Bulkhead Letters Sushi 250mm","Sign Bulkhead Letters Salad 250mm","Sign Bulkhead Letters Juice 250mm","Sign Bulkhead Letters Hot Food 250mm","Sign Bulkhead Letters Street Food 250mm","Sign Bulkhead Letters Hot Soup 250mm","Sign Bulkhead Letters Fresh To Go 250mm","Sign Bulkhead Letters Chilled 400mm","Sign Bulkhead Letters Cafe 400mm","Sign Bulkhead Letters Bakery 400mm","Sign Bulkhead Letters Deli 400mm","Sign Bulkhead Letters Meat 400mm","Sign Bulkhead Letters Dairy 400mm","Sign Bulkhead Letters Milk 400mm","Sign Bulkhead Letters Frozen 400mm","Sign Bulkhead Letters Fruit&Veg 400mm","Sign Bulkhead Letters Sushi 400mm","Sign Bulkhead Letters Salad 400mm","Sign Bulkhead Letters Juice 400mm","Sign Bulkhead Letters Hot Food 400mm","Sign Bulkhead Letters Street Food 400mm","Sign Bulkhead Letters Hot Soup 400mm","Sign Bulkhead Letters Fresh To Go 400mm","SignB/head Letters Chilled 2mmPVC 250mmH","SignB/head Letters Cafe 2mmPVC 250mmH","SignB/head Letters Bakery 2mmPVC 250mmH","Woolworths Ltd Logo","SignB/head Letters Deli 2mmPVC 250mmH","SignB/head Letters Meat 2mmPVC 250mmH","SignB/head Letters Dairy 2mmPVC 250mmH","SignB/head Letters Milk 2mmPVC 250mmH","SignB/head Letters Frozen 2mmPVC 250mmH","SignB/headLetters Fruit&Ve 2mmPVC 250mmH","SignB/headLetters Chilled 2mmPVC 400mmH","SignB/headLetters Cafe 2mmPVC 400mmH","SignB/headLetters Bakery 2mmPVC 400mmH","SignB/headLetters Deli 2mmPVC 400mmH","SignB/headLetters Meat 2mmPVC 400mmH","SignB/headLetters Dairy 2mmPVC 400mmH","SignB/headLetters Milk 2mmPVC 400mmH","SignB/headLetters Frozen 2mmPVC 400mmH","SignB/headLetters Fruit&Ve 2mmPVC 400mmH","Sign Milk Paint \\(on demand\\)","Sign Hanging Cafe w/Capping","Sign Hanging Bakery w/Capping","Sign Hanging Deli w/Capping","Sign Hanging Meat w/Capping","Sign Hanging Dairy w/Capping","Sign Hanging Milk w/Capping","Sign Hanging Chilled w/Capping","Sign Hanging Frozen w/Capping","Sign Hanging Fruit&Veg w/Capping","Sign Hanging Sushi w/Capping","Sign Hanging Salad w/Capping","Sign Hanging Hot Food w/Capping","Sign Hanging Hot Soup w/Capping","Sign Hanging Fresh To Go w/Capping","Sign Hanging Checkouts w/Capping","Sign Aisle Thomas Dux w/Capping","Sign Street Thomas Dux 283mmWx80mmH","Data Insert Thomas Dux 914mmWx26mmH","Sign End Thomas Dux 815mmWx300mmH","Kit Fin's Thomas Dux \\(12 Types\\)","Sign Aisle w/Capping 1-10","Sign Category Insert","Kit Sign Arial w/Capping \\(3 Signs\\)","Kit Sign Hanging Cafe Order Here","Kit Sign Hanging Cafe Collect Here","Kit Sign Hanging Bakery \\(2 Signs\\)","Kit Sign Hanging Deli \\(2 Signs\\)")
  
  , MaterialMaster_FOOD_1501_1600 = c("Kit Sign Hanging Milk Bar \\(2 Signs\\)","Kit Sign Sushi \\(2 Signs\\)","Rail Bulkhead 1400mm","Rail Bulkhead 1200mm","Header Milk4600Wx100mmHx82mm text","Sign Vittoria Coffee 400mm Counter POS","Sign Vittoria Coffee 250mm Counter POS","Sign Vittoria Coffee 400mm Side Counter","Sign Vittoria Coffee 250mm Side Counter","Frame Column 965mmHx450mmW","Kit Magnetic Cafe Prints 915x400mm","Kit Graphics Column 915x400mm","Sign Opening Hours A3","Flags Now Open 300x70cm","Sign Frame End 700w","Kit Sign Dinner Deal Fridge Gondola","Frame Gondola End","Kit Sign Gondola End","Sign Counter A4","Sign A4 Dump Bin Table \\(Portrait\\)","Sign A4 Dump Bin Table \\(Landscape\\)","Kit Sign Specials A4 \\(24 Signs\\)","Kit Inserts Info A4 \\(8 Types\\)","Kit Inserts Catchy A4 \\(9 Types\\)","Sign Street 283mmWx80mmH","Header Category Frozen & Chilled","Sign A Frame \"All Day Cafe Deals\"","Sign A Frame \"You Asked Me & We Listened","Sign A Frame \"Whats in Store\"","Sign A Frame \"New Milk Bar\"","Sign A Frame \"Thomas Dux\"","Sign Welcome","Sign \"Thank You\"","Frame End Produce Gondola","Sign Produce Gondola End","Sign Wobblers 70mmWx100mmH","Kit Sign Back of House \\(7 Signs\\)","Sign Insert Kilojoule","Sign Insert Organic","Sign Insert Seafood","Data Stripping Green Organics","Sign Thank You \\(Space Filler\\)","Signage Florist","Sign \"Too Good to Last Sorry\"","Ticketing Cafe","Form Feedback A5","Decal Take Me 700mmWx100mmH","Sign Trolley Bay","Stand Ticket Tall","Clips","Cards Coffee Loyalty","Sign A Frame \"Groceries & Meals\"","Sticker Woolworths Limited \\(Window\\)","Kit Hot Food Sign","Sign Souvenir","Lunch to go Shelf Limter Box","Lunch to go A-Frame","Lunch to go Landscape Counter Card Offer","Lunch to go Landscape Counter Card -Hero","Lunch to go A4 Counter Card Whole Offer","Lunch to go A4 Counter Card Step 1 Hero","Lunch to go Fridge Top Header Panel","Lunch to go Fridge Fin","Lunch to go Extrusion for fridge fin","Lunch to go Side kick Unit sign","Lunch to go Basket Cage data insert","Lunch to go Basket Cage data Strip","Lunch to go Product Sticker","Lunch to go Wobbler","Lunch to go Data insert 912w x 25h","Lunch to go Side kick Unit Data insert","Lunch to go Fridge Strip Decal","Kit Soup Sign 100x70mm \\(12signs\\)","Kit Soup & Bread Roll A4 \\(4 Signs\\)","Kit Coffee Large A4 \\(4 Signs\\)","Kit Coffee & Croissant A4 \\(4 Signs\\)","Kit Coffee&Croissant data insert 30x275","Soup & Roll data insert 30x275 \\(4 signs\\)","Counter Service Corner Unit W/Slatwall","Drawer Cigarette Unit - Front Counter","Drawer Cigarette Unit Rear Counter","Carton Unit Rear Counter","Surround Unit Ambient 1215x900mm","Station Soup Inline Counter 1455mm","Station Soup Inline Counter 1013mm","Kiosk SS90 Satin Black LH","Kiosk SS90 Satin Black RH","Buffer Floor 66x66 Stained H/Wood","Cigarette drawers 915x700x1430","Shelf Custom Black","Shelf Custom Grey","Pallet Mover Euro","Box Suggestion","Hose Reel S/S Retractable 5m","Safe Generation 14 Intelli Vebd TDR","Alarm Box","Remote Alarm","Hand Dryer Dyson Airblade AB14 Grey","Melamine Shelf 1810x415mm Narrow Fixture","Post T 44Notch 210x430x430 L/Leg S/Black")
  , MaterialMaster_FOOD_1601_1700 = c("Post L 44Notch 2010x430 L/Leg S/Black","Panel Punch Metal 902x450mm S/Black","Panel Punch Metal 902x327mm S/Black","Cover Top Cap 915mm S/Black","Post L 1060x430 L/Leg S/Black","Post T 44Ntch 2010x430x430 L/Leg P/Pearl","Joiner Inter 355 Narrow Fixture P/Pearl","Post L 44Ntch 2010x430 L/Leg P/Pearl","Post T 1450x430x430 L/Leg L/Prof P/Pearl","Post L 1450x430 L/Leg L/Prof Pewt/Pearl","Shelf Base 913x382mm W/Buffer","Divider 280x100mm for Narrow Base Shelf","Basket Universal 912x332mm Narrow Fixt","Divider 350x100mm for Narrow Basket","Divider Centre Pillow Suit Narrow Shelf","Divider End Pillow Suit Narrow Shelf","Shelf Mid Wire 913x364mm Narrow Fixture","Shelf Base Wire 913x382mm Narrow Fixture","Basket 600x316x100x50 Narrow Fixture","Shelf Base 600x336 Narrow Fixture","Shelf Mid 600x336 Narrow Fixture","Shelf 700mm for Narrow Dense End","Basket 700mm for Narrow Dense End","Basket Bread 914x399mm Narrow Fixture","Divider 399mm for Narrow Bread Basket","BRACKET 4 NOTCH 355MM SEMITD","Bracket 4 Notch 355mm W/Satin","Bracket 4 Notch 355mm Pewter Pearl","Bracket 305 Sloping Basket Pewter Pearl","Bracket 305 Sloping Basket S/Black","Shelf Spice 913x346 Slip Coat","Plug Half - End Post 40x20mm","Post T 2100X535X535mm Metro BOW","Buffer Rail 874mm BOW","Protector Leg Black","Post Plugs Metro BOW","Post Plug Metro Black","Bumper 1830 for Bolt on Outrigger LegBOW","Bumper 915 for Bolt on Outrigger LegBOW","X Bars 450mm L x 210mm D White","Flipscan Rail for 450mm X Bar","Shelf Mid Info w/Acrylic Cover Cosmetic","PelmetCos/Skin/Sun 450x914incl\\(2\\)Bracket","Pelmet Metal End Skin/Sun 450x914mm LH","Pelmet Metal End Skin/Sun 450x914mm RH","Shelf Base Cos 914x490 W/Acrylic Sleeve","Bracket Shelf for 365mm Shelf WhiteSatin","Pelmet Metal Return End 300x914mm HBA LH","Pelmet Metal Return End 300x914mm HBA RH","Shelf wire 914x400mm Chrome","Bracket for 400mm shelf SatinWhite","Pelmet Metal HBA 300x914mm w/2 Brackets","X Bars 450mm L x 100mm D White","Shelf Base Skin/Sun 914mm x 490mm","Shelf Steel 912x310mm HBA Satin White","Bracket for 310mm shelf Satin White","Shelf Steel 912x400mm HBA Satin White","Shelf Steel 912x468 HBA Satin White","PlugPost Top Cover Cap L/Profile W/Satin","Panel Punch Metal 902x450 W/Satin","Panel Punch Metal 905x327 White Satin","Cover Top Cap 915mm W/Satin","Panel Plain Metal 600x450mm W/Satin","Panel Punch Metal 600x450 W/Satin","Buffer Rail 915mm White Satin No P/Plug","Buffer Rail 1830mm White Satin No P/Plug","Rail Buffer 600mm White Satin No P/Plug","PostT Mid 1387x540x540 L/Leg W/Satin","Post Stiffner","Leg Extension 750mm Pewter Pearl","Cover Top Cap 915mm APO","Pelmet Metal 300x914mm P/Pearl LH End","Pelmet Metal 300x914mm P/Pearl RH End","PelmetMetal 300x914mm P/Pearl w/2Bracket","Plug Post Cover Cap 40x40 Black","Plug Post Cover Cap 40x40 White","LEG EXTENSION 750MM SEMITD","PANEL PUNCH METAL 902X450 SEMITD","PANEL PLAIN METAL 902X450 SEMITD","PANEL PUNCH METAL 905X327 SEMITD","POST L 1060X611 L/LEG L/PROF SEMITD","COVER TOP CAP 915MM SEMITD","PANEL PUNCH METAL 1/4 228.60X450 SEMITD","PANEL PUCH METAL 1/2 457.20X450 SEMITD","PANEL PUNCH METAL 3/4 685.80X450 SEMITD","PANEL PLAIN METAL 600X450MM SEMITD","PANEL PUNCH METAL 600X450 SEMITD","POST T MID 1387X540X540 L/LEG SEMITD","L POST REINFORCED 44 NOTCH SEMITD","Panel Punch Metal 228x450mm P/Pearl","Panel Punch Metal 457x450mm P/Pearl","Panel Punch Metal 685x450mm P/Pearl","Plug Post Top Cover Cap L/Profile APO","Post L/Leg Ext 310x120x40 P/Pearl HD900","Beam Rear 910x50x30 P/Pearl HD900HD900","Beam Rear 1825x50x30 P/Pearl HD900HD900","Beam Front 1822x50x30 P/Pearl HD900HD900","Joiner Top Outrigger 810 P/Pearl HD9000","Joiner Intermediate 810 P/Pearl HD90000","Shelf Melamine 1810x840x16mm HD900HD900")
  , MaterialMaster_FOOD_1701_1800 = c("Beam Front 907x50x30 P/Pearl HD900HD901","Shelf Melamine 905x840x16mm HD900 HD900","Panel Plain Metal 902x450 PewterPearl","Panel Punch Metal 902x450 PewterPearl","Cover Top Cap 915mm Pewter Pearl","Panel Plain Metal 600x450mm P/Pearl","Panel Punch Metal 600x450 Pewter Pearl","Post T 44Ntch 2010x540x540 L/Leg P/Pearl","Post L 44Ntch 2010x540 L/Leg P/Pearl","Post Outrigg Full 44Ntch 1855x40 P/Pearl","Rail Buffer 915mm Pewter Pearl","Rail Buffer 1830mm Pewter Pearl","Post L 1060x611 L/Leg L/Prof P/Pearl","Side Panel Kit 1830x495 44Ntch P/Pearl","Panel Punch Metal 905x327 Pewter Pearl","Beam Front 1785x40x40 Pewter Pearl","Pelmet Hero Metal 1830mm Pewter Pearl","Pelmet Hero Metal 915mm Pewter Pearl","Post T Mid 1387x540x540 L/Leg P/Pearl","Rail Buffer 600mm Pewter Pearl No P/Plug","Rail Buffer 1830 Pewter Pearl No P/Plug","Rail Buffer 915 Pewter Pearl No P/Plug","Beam Front 875x40x40 Pewter Pearl","Post T Mid 1640x540x540 L/Leg P/Pearl","Brackets LH/RH Pelmet 390mm P/Pearl","Rail Buffer 915 White Satin","Rail Buffer 1830 White Satin","Rail Buffer 1515mm White Satin \\(AFM\\)","Front Wire 914x100mm Chrome","Divider Chrome 455x102mm for Base Shelf","Basket Uni 914x405x100x45mm Non Snag Chr","Divider Basket 406x100x45mm Variety Chr","Shelf Mid Wire 915x470mm Chrome","Shelf Base Wire 915x470mm Slope Front Ch","Bracket White to suit 400mm Shelf","Shelf Steel 470x914mm White My Kitchen","Brackets to suit 470mm White Shelf","Shelf Steel 310x914mm White My Kitchen","Brackets to suit 310mm White Shelf","Shelf Steel 400x914mm White My Kitchen","Shelf Base Wire 600x470mm Slope Front Ch","Shelf Mid Wire 600x470mm Chrome","Front for Wire Shelf 600x100mm Chrome","Basket Uni 600x406x100x45mm Non Snag Chr","Divider Centre Pillow Chrome","Divider End Pillow Chrome","Shelf Base 750mm Chrome","Shelf Mid 750mm Chrome","Bracket 750mm Pewter Pearl","Basket Bread 912mm Chrome","Divider 458mm Spice Shelf Chrome","Shelf Confectionary 250mm Deep","Shelf Confectionary 330mm Deep","Bracket Incline 15 Deg Fixed 250mm","Bracket Incline 15 Deg Fixed 330mm","Divider Wire Confectionary Shelf 250mm","Divider Wire Confectionary Shelf 330mm","Shelf Confectionery 455mm Chrome","Bracket Confectionery 455mm Pewter Pearl","Bracket Confectionery 330mm Pewter Pearl","Shelf Mid Wire 915x510mm Chrome","Base Shelf 915x510mm Chrome","Universal Basket Divider 527x100x45mm Ch","Spice Shelf 909x527 Pewter Pearl","Bracket ADJ 510x41P Pewter Pearl","Base Shelf Support 800x450x120mm PP","Shelf Divider 483x102mm Chrome","Universal Basket 915x510mm Chrome","Bread Shelf 915x510mm Chrome","Base Shelf Wire 600x510","Shelf Wire 600x510","Basket 600x510mm","1/4 Bay Base Shelf Wire 228.6x510","1/4 Bay Shelf Wire 228.6x510","1/2 Bay Base Shelf Wire 457.2x510","1/2 Bay Shelf Wire 457.2x510","3/4 Bay Base Shelf Wire 685.8x510","3/4 Bay Shelf Wire 685.8x510","Shelf Book Type Chrome","Divider for Book Shelf Chrome","Bracket 534x124mm HBA White","Shelf Bread 600mm","1/2 Bay Capping 457.2x510","3/4 Bay Capping 685.8x510","1/4 Bay Capping 228.6x510","Bracket Mod Capping 460 S/Blk Pair LH/RH","Bracket Mod Capping 510 S/Blk Pair LH/RH","Shelf Basket Capping 600mmx330mm","Shelf Basket Capping 913mmx330mm Chrome","Bracket Capping W/Satin 307mm","Bracket Capping P/Pearl 330mm","Bracket 6 Notch 470mm W/Satin","Shelf Mid Wire 913x766 Chrome HD900D900","Shelf Base Wire 913x766 S/Front Ch HD900","BRACKET 750MM SEMITD","BRACKET ADJ 510X41P SEMITD","BASE SHELF SUPPORT 800X450X120MM SEMITD","BRACKET 5 NOTCH 405MM SEMITD","SPICE SHELF 909X527 SEMITD","BRACKET CONFECTIONERY 330MM SEMITD")
  , MaterialMaster_FOOD_1801_1900 = c("BRACKET CONFECTIONERY 455MM SEMITD","BRACKET 510MM 3MM BODY SEMITD","POST T REINFORCED 44 NOTCH SEMITD","Bracket 470mm SemiTD","Bracket 5 Notch 405mm Pewter Pearl","Bracket 6 Notch 470mm Pewter Pearl","Frame End Headerboard Double L/Hgt P/Prl","Shelf Spice 914x458 Slip Coat Grey","Protector Low Leg Black","Data Strip 1225x30mm Charcoal Dairy/Meat","Holder Ticket Adj Clear Plastic 35x30mm","Data Strip 1200x30mm Clear w/Adhesive","Holder Signage Deli Bowl","End Clips Plastic for Bracket Melamine","TapeSelf/AdhesiveNeopreneP/Table15m Roll","Bull Nose for wire shelf 30x915mm","Clips for Bullnose 4.0 Clear","Rail for wire shelves 915mm","Divider breakable 60mm Clear","B/Nose metalshelf w/adhesive tape 30x915","Rail w/adhesive for metal shelf 915mm","Stand A2 Fresh Market Update","Top Rail P/Pearl for Collapsible End LVA","Draw Runner L/R White","Kit Hook 150mm Beauty Accessories","Hook Flipscan 100mm Fastback","Drawer Front Small BLACK \\(pack of 10\\)","Tray Top Cover Cig Pushfeed BLACK","Universal Drawers WHITE pack of 4","Pushfeeds for Cig Draws pack of 24","Kit Pushers shelf ethnicity","Hook Technology","Kit Hooks Front End Merchandiser LVA","Hook GA Gantry Hanging Sign PK10 Black","Hook Spacer Gantry Sign PK50 Black","Stand Dinner Made Easy","Holder Sign Supergrip Pk20","Data Strip Package 40mm 1/4 Bay","Data Strip Package 40mm 1/2 Bay","Data Strip Package 40mm 3/4 Bay","Hook Single Prong 75mm Fastback","Fastback HD White","Fastback HD Grey","Hook Double Prong 300mm Plastic Dipped","Hook Single Prong 155mm Fastback","Hook Flipscan 240mm Fastback","Hook Single Prong 225mm Fastback","Hook Flipscan 255mm Heavy Duty Bar","Fastback Grey","Hook Single Prong 51mm Mesh FBck BEMSpin","Hook Flipscan 55mm wMesh FBack BEM Spin","Flipscan 215mm for BEM Spinner Signage","Flipscan 75mm White Satin","Hook 150mm xbar fastback white","Flipscan 450mm notched white","Hook White 312mm, Antisweep single","Hook White 312mm, Antisweep w/flipscan","Hook Antisweep W/Flipscan 200mm","Hook Single 12\" Grey Heavy Duty","Hook Flipscan 12\" Grey Heavy Duty","Hook Single 12\" White Heavy Duty","Hook Flipscan 12\" White Heavy Duty","Hang Clips for Mesh Backboard Notre Dame","Crossbar Extended 915xmm Chrome","HookSingle Crossbar 230mm Chrome","Hook Flipscan Crossbar 230mm Chrome","Hook Flipscan 240mm Fastback White","Hook Single Prong 225mm Fastback White","Fastback Light Duty Whte","Hook Single Prong 100mm Fastback","Security Hook Shaving","Crossbar Shaving","Wine Barrel Half 800H","Wine Barrel Full 800H","Produce Stock Trolley 1211W x 1400Hmm","End Collapsible1975mm W/Shelves&PP Frame","Bracket Adaptor End Collapsible 47Ntch","Display Acrylic Dinnerware","Shelf Front Acrylic My Kitchen","Table Demonstation Sushi Incl. Signage","Frame P/P BEM Collapsible End","Display Unit Razor Blades \\(1 Bay\\)","Display Unit Oral B","Bracket Toothbrush w/16hook Narrow White","Bracket Floss w/17 hooks Narrow White","Acrylic Display A5-4 Magnets My Kitchen","PanelAcrylic900x450Wht 3Mag/StripM/Kitch","Merch Sushi CompW/Shelves1600x1125x350","Kit Bumper & Brace Suit D Pallet","Kit Wedge & Brace Suit D Pallet","Hook Scan Assy for Broom & Mop Bracket","Roller shelf 470 H/Col C/W adapter brkt","Roller shelf 490 H/Col C/W adapter brkt","Display Box Hair Care Acrylic","R/Shelf 913x365 W/Frnt riser/11Div/Brkts","R/Shelf 913x400 W/Frnt riser/11Div/Brkts","R/Shelf 913x470 W/Frnt riser/11Div/Brkts","R/Shelf 913x490 W/Frnt riser/11Div/Brkts","Rack Plate Display","EndBay Kit M/Height Fixutre1150x1420 Wh")
  , MaterialMaster_FOOD_1901_2000 = c("End Bay Kit M/Ht Fix 5 Shlf 1150x1420Wht","EndBay Kit M/Ht Fix 5 Shlf 1150x1420 P/P","Feature End HBA 1385&S/HangSellPanels PP","FeatureEnd HBA1385&S/HangSellPanels S/WH","Pallet Guard 7\" Black, Adjustable","Pallet Surround Yellow","Base Pallet Extension 915x250x190 Red","Frame Odd Bunch Black","Frame Odd Bunch Semi TD","BEM 2001 w/Shelves & Price Point Frame","BEM 1400 w/Shelves & Price Point Frame","BEM 2001 wShelves,PP Frame,Cutout Canopy","Magazine Pocket Ctn of 2 for slat panel","Magazine Display Unit 2 Bay 1828","Magazine Display Unit 3 Bay 2742","Divider Acrylic Magnetic for Magazine Un","End Collapsible 1975mm w/large shelves","Shelf Mesh Side 407x223x85 for EQTT3300","Promo End with Cupboard SemiTD","Promo End SemiTD","Promo End with Cupboard P/Pearl","Promo End P/Pearl","Promo End Joiner SemiTD","Promo End Joiner P/Pearl","Kit Bulkfood Hoppers Suit 2 Bay","Kit Bulkfood Hoppers Suit 5 Bay","Merch Chicken Warmer MK5 BAKERY","Shelf with Frame Promo End SemiTD","Shelf with Frame Promo End P/Pearl","Bracket Promo End SemiTD","Bracket Promo End P/Pearl","Promo End Wire Shelf Stopper","Kit Mops & Brooms 400mm P/Pearl","Pelmet Health Food Start Bay","Pelmet Health Food Add Bay","Tool Datarail Fixing \\(5 per bag\\)","Data Strip & Rail Flat \\(Mid Shelf\\) 915mm","Data Strip & Rail Angle\\(BaseShelf\\) 915mm","Divider Acrylic 471 x 60mm","Divider Acrylic 471 x 120mm","Riser Front 100x40mm","Clip to secure Front Riser to Shelf","Divider Acrylic 365 x 60mm narrow","Kit Mops & Brooms 400mm SemiTD","END COLLAPSIBLE 1975MM W/LGE SHLF SEMITD","KIT 1/2 BAY INFILL 457.2X510 SEMITD","KIT BOOK\\VIDEO 1 BAY COMPLETE SEMITD","BEM 2001 WSHELVES & P/POINT FRAME SEMITD","KIT 3/4 BAY INFILL 685.8X510 SEMITD","KIT 1/4 BAY INFILL 228.6X510 SEMITD","ENDBAY KIT M/HT 5 SHLF1150X1420 SEMITD","MAGAZINE DISPLAY UNIT 2 BAY 1828 SEMITD","MAGAZINE DISPLAY UNIT 3 BAY 2742 SEMITD","Shelf Base Metal Timer Front S/White","GANTRY NO LIGHT SEMITD HEALTHFOOD","SHELF BASE MID BAY TIMBER FRONT SEMITD","PANEL PLAIN 470X915 W/SATIN","PANEL PLAIN 327X915 W/SATIN","Kit 1/4 Bay Infill for Column 228.6x510","Kit 1/2 Bay Infill for Column 457.2x510","Kit 3/4 Bay Infill for Column 685.8x510","Kit Bar Clothes Hanger w/Hooks","Shelf Medicinal Security","Kit Divider Low Medicinal Security","Kit Divider Med/Tall Medicinal Security","Kit wedge and bumper 510mm SemiTD","Kit wedge and bumper 510mm P/Pearl","Kit 1/4 Bay Infill 228x465mm P/Pearl","Kit 1/4 Bay Infill 457x465mm P/Pearl","Kit 3/4 Bay Infill 685x465mm P/Pearl","Graphic Middle Shelf Select&Nail 906x345","Cross Bars for Health&Beauty 1150","Crossbar Universal Set Heavy Duty","Start Bay Outrigger Health Food Full","Add Bay Outrigger Health Food Full","Start Bay Outrigger Health Food 3/4","Add Bay Outrigger Health Food 3/4","Start Bay Outrigger Health Food 1/2","Add Bay Outrigger Health Food 1/2","Start Bay Outrigger Health Food 1/4","Add Bay Outrigger Health Food 1/4","Kit Book\\Video 1 Bay Complete Pewt/Pearl","Display Broom/Mop 3 Double Prong P/Pearl","Display Mop Prong \\(Single\\) PPearl","Hook Scan 400x90mm 8mm dia P/Pearl","Hook Double Prong 400x90mm dia 8 P/Pearl","Stabilizer Double Prong dia 8mm P/Pearl","Hook Scan 400x90mm 8mm dia SemiTD","Hook Double Prong 400x90mm dia 8 SemiTD","Stabilizer Double Prong dia 8mm SemiTD","Pelmet DVD 915x260mm Magnetic","Holder Tbar StreetArm HBA 655x270W/Satin","Holder Tbar Street Arm 655x270 S/Black","Scrum Unit w/channels 910x1650 HBA S/Wh","Pel/Graph Womens Hair Removal Purple","ARM FOR STREET SIGN GROCERY SEMITD","Pelmet graphic baby nappies 300x914 Pur","Pelmet graphic Dental Care 300x914mm Pur","Pel/graphic Mouth wash 300x914mm Pur","Pel/graphToiletry travelpacks300x914 Pur")
)

# sapply(MaterialMaster_FOOD_LIST, function(x) {
#   npiped <- paste0(x, collapse="|")
#   FileData_ALL$Rule_Food[grep(npiped, FileData_ALL$Asset_description, ignore.case = TRUE)]  <- 'Food'
# })

for(i in 1:length(MaterialMaster_FOOD_LIST)){
  x <- MaterialMaster_FOOD_LIST[[i]]
  npiped <- paste0(x, collapse="|")
  which_no_rule_yet <- which(FileData_ALL$Rule_Food == '' & FileData_ALL$Rule_BWS == '')
  if(length(which_no_rule_yet)<1){break}
  which_match <- grep(npiped, FileData_ALL$Asset_description[which_no_rule_yet], ignore.case = TRUE)
  idx_match <- which_no_rule_yet[which_match]
  FileData_ALL$Rule_Food[idx_match]  <- 'Food'
  print(i)
}

# saveRDS(FileData_ALL,"C:/USERDATA/njcy8/My Documents/Projects/ED - Swan Data/Output/FileData_ALL_backup.rds")

#Rule 6.2 - Asset Descriptions that contains specified list of BWS Materials  = BWS
#===============================================================================================

MaterialMaster_BWS_LIST <- list(
  MaterialMaster_BWS_001_100 = c("Slat Wall Hook 150mm","Unit START MXW10&13 PrWine 1324x2280 Blk","Kit Impulse Shelving Counter AddBay","Craft Beer Wire Shelf 1258 x 640mm","Display Unit Non Liquor","Display Unit Beer Multipack Carrier","Holder Price A3 Double Perspex","Cash Tube/Data/Power Pole Round Black","Pole Cash Vac H/Grey\\(Height as per plan\\)","Pin Board Beaten 1800x1000","FGT Spirit Gondola End 1774x1078x318mm","Hand Basket - Black & BWS Logo PK20","Stand for Hand Basket","Roller Basket DM","Liquor Trolley W/Basket-DM","Liquor Trolley W/Basket Trav Whls - DM","Trolley Customer W/Baby Capsule DM","Tasting Unit 8 Bottle Enomatic Elite2","Kit Cleaning Standard Dan Murphys","Small Security Bottle Cap \\(RF EAS Gates\\)","Large Security Bottle Cap \\(RF EAS Gates\\)","Neck Tag - Security \\(RF EAS Gates\\)","Mirror Security 45cm Convex","Small Security Bottle Cap \\(Overhead EAS\\)","Large Security Bottle Cap \\(Overhead EAS\\)","Neck Tag - Security \\(Overhead EAS\\)","Detacher Magnetic \\(Overhead EAS\\)","Kit Alpha Security Caps & Detachers DM","Alpha Security Detachers DM","Spittoon Wine Tasting","Barrel Wine Half w/Logo & Black Hoops","Barrel Wine Full w/Logo & Black Hoops","Forklift Crown LPG incl Gas Bottle","Registration Crown Rideon Lift Truck NSW","Registration Crown Rideon Lift Truck VIC","Registration Crown Rideon Lift Truck TAS","Registration Crown Rideon Lift Truck ACT","Registration Crown Rideon Lift Truck SA","Registration Crown Rideon Lift Truck NT","Registration Crown Rideon Lift Truck WA","Registration Crown Rideon Lift Truck QLD","Registration Crown Rideon Lift Truck NSW","Sign Wall Capping Hook Over 1283x237","Sign Category D/S W/Integration 430x140","Lightbox w/brackets XLarge & Insert","Sign Lightbox XLarge Insert 2648x684","Lightbox w/brackets Narrow DM","Sign Lightbox Narrow Insert 1265x220","Lightbox Power Supply 12V","Sign Light Box Small Insert 1265x440","Sign Light Box Large Insert 1775x440","Lightbox W/Brackets Large DM","Lightbox W/Brackets Small DM","Sign Wall Capping Hook Over 1283x440","Sign Category Horizontal Beer 1280x380","Sign PVC Wanzl & Stand 1800x450","Sign Category Island Dairy BayM 1250x220","Sign Under Security Cabinet 1280x120","Post L 1500x535mm Satin Black","Shelf Melamine Black 565x430mm","Shelf Metal 1166x505mm Satin Black","Panel End Metal Satin Black","Holder Sign Low Profile","Frame A Entry 1120x630mm DM","Wine Cabinet Vintec V190 RH Hinge","Wine Cabinet Vintec V190 LH Hinge","Vintec Wine Cabinet Metro Freight","Vintec Wine Cabinet Regional Freight","Shelf Mid Wire 913x364mm Narrow Sat Blk","Shelf Base Wire 913x364mm Narrow Sat Blk","Shelf Wire Mid 915x600mm Sat Black","Shelf Wire Base 915x600mm S/Black Slant","Bracket 600mm Sat Black","Shelf Metal 610x505mm Satin Black","Display Wine Tasting Stand","StandTwin Liquor Service&Screw NCRMonito","StandTwin Liquor Service&Screw NCRMonito","Post L 1500x535mm Satin Black","Panel Plain 450x902mm Satin Black","Board Feature End Black 200x16x948mm 1A","Board Black 200x16x1846mm 2A","Board Black 200x16x1826mm 3A","Board Black 200x16x913mm 4A","Bracket 470mm 6 Notch Satin Black","Post T 1469x470mm","Post L 1469x400mm Narrow","Shelf Base 913x400mm Narrow","Bracket ADJ 355mm Satin Black","Beam Light Duty 1135x45mm Satin Black","Beam Light Duty 535x45mm Satin Black","Post H Frame 2400x505mm Satin Black","Post H Frame 1800x505mm Satin Black","Shelf Melamine Black 1166x42mm","Shelf Melamine Black 565x430mm","Shelf Metal 1166x505mm Satin Black","Panel End Metal Satin Black","Holder Sign Bulk Display","Holder Sign Low Profile","Coolroom Racking Start Bay 2400mm","Coolroom Racking Add Bay 2400mm")
  , MaterialMaster_BWS_101_200 = c("Coolroom Racking Start Bay 1200mm","Coolroom Racking Add Bay 1200mm","Frame \'X\' 800x1800mm","Frame A Entry 1120x630mm DM","Unit Y Start Bay suit A1 MK2 Fine Wine","Unit End HalfRound suit A1 MK2 Fine Wine","Unit End Half Round suit AT Fine Wine","Lock Box Trading Hour DM","Frame End FGT 1804x400mm","Kit FGT Shelf1005x332x22&Beam End 1012mm","Stand Half Wine Barrel Angled","FGT Spirit Gondola End 1774x1078x318mm","Display Unit Non Liquor","Unit STRT45d\"\"MC\"\"Book2280x823x584 Blk","Unit STRT AT MK3 F/W 1314x552x1478","Unit START A1 MK2 F/Wine 1314x403Dx1244H","Unit ADD A1 MK2 F/Wine 1298x403Dx1244H B","Shelf Lay Down FGD/FGT 195x1258x467","Beam Stabilising 1258mm","Lock Box Triple Shelf 1245x1200 LED","Box Lock Double Shelf 1245x754.6 LED","Box Lock Single Shelf 1245x360x360 LED","Shelf Wire 1258x467x22mm","Frame H D/S 1804x1078mm","Frame H D/S 1518x1078mm","Shelf Curved 1255x597mm","Beam Adjustable1282mm","Bumper Rail 1280 Adjustable 1280mm","Panel Side W/Clip Top 321x1010mm","Panel Side W/Clip Botom 531x1250mm","Frame H Single 2280x539mm","Beam End 1258x115mm","Kit Punched Panel W/joiners 1271x682mm","Craft Beer Wire Shelf 1258 x 640mm","Display Unit Beer Multipack Carrier","Holder Price A3 Double Perspex","Bottle Holder Small - Fine Wine","Unit End FGT 1078Xx400Dx1804H Black","Kit End Shelf Curve Suit FGD NarrowUnits","Bottle Holder Acrylic DM","Beam Bottom FGD/FGT 1258mm","Beam Bottom Adjustable FGD/FGT 1258mm","Bumper Rail FGD/FGT 1285mm","Frame FGD Extension Single 571x624x32","Unit START B PrWine 2280x1832x768 Blk","Unit ADD B PrWine 2280x1832x768 Blk","Kit B Unit Bottle Holder 9 Column 110mm","Kit B Unit Bottle Holder 9 Column 65+95m","Kit B Unit Shelf and Beams Assembly Blk","Kit B Unit Magnum Bottle Holder Blk","Table Staff Lunch/Train 1200x750x755","Bar Stool Wooden","Wood Top - Bar Stool Wooden","Board Beer Signage U Shape DM","Sign FG Units adjT-Bar Hang Panel Blk","Beer Board Rotating Single","Beer Board Rotating Double","Frame A3 Portrait Snapframe Black","Snapframe A3 Black UV","C/O Exp I/Safe M/Black 1087x757x900 LH","C/O Double I/Safe M/Black 2600x2100x900","CO Single I/Safe M/Black 1416x2100x900RH","Counter Miniatures M/Black 950x605x1065","Unit Open WBag Pack 477Wx628Dx868H G/Oak","Corner Void 520Wx628Dx868H G/Oak LH","Unit Storage Return 878Wx480Dx868H G/Oak","Unit Cash Vac 520Wx483Dx868H G/Oak","Unit CigDraw Liquor 918Wx430Dx868H G/Oak","Unit High GI/Shelf 918Wx280Dx1332H G/Oak","Unit High Circle 918Wx280Dx1332H G/Oak","Unit Low GlassShelf 918Wx280Dx532H G/Oak","Unit Reg Liquor 600Wx628Dx868H G/Oak","Unit Front Infill 600Wx628Dx868H G/OAK","Benchtop Front 32Hx898DxLength to suitLH","Roll Cage Liquor Security","Kit Liquor C/Out Wire Basket&Bag Holder","Bar Stool Wooden","Bar Stool Removable Cushion","Safe Gen 7 Intelli Duel DM","Stand Enviro Bag","Data Strip with Hangers39x700mm Blk PK20","Data Strip Hinged 39x1250mm Clear PK20","Data Strip Hinged 39x1140mm Black PK20","Data Strip 39x1220mm Cold Temp Blk PK20","Data Strip Hinged 39x915mm Black PK20","Data Strip Hinged 39x915mm Clear PK20","Data Strip Hinged 39x1140mm Clear PK20","Chair Aim Gas Lift Bar Stool","Ladder Safety 3 Step","Shelf Mid Wire 913x364mm Narrow Sat Blk","Shelf Base Wire 913x364mm Narrow Sat Blk","Shelf Metal 610x505mm Satin Black","StandTwin Liquor Service&Screw NCRMonito","Post L 1500x535mm Satin Black","Panel Plain 450x902mm Satin Black","Board Feature End Black 200x16x948mm 1A","Board Black 200x16x1846mm 2A","Board Black 200x16x1826mm 3A","Bracket 470mm 6 Notch Satin Black","Post T 1469x470mm")
  , MaterialMaster_BWS_201_300 = c("Post L 1469x400mm Narrow","Shelf Base 913x400mm Narrow","Bracket ADJ 355mm Satin Black","Beam Light Duty 1135x45mm Satin Black","Beam Light Duty 535x45mm Satin Black","Post H Frame 2400x505mm Satin Black","Post H Frame 1800x505mm Satin Black","Shelf Melamine Black 1166x42mm","Shelf Melamine Black 565x430mm","Shelf Metal 1166x505mm Satin Black","Panel End Metal Satin Black","Holder Sign Low Profile","Unit Y Start Bay suit A1 MK2 Fine Wine","Frame End FGT 1804x400mm","Frame End FGT 1804x400mm","Kit FGT Shelf1005x332x22&Beam End 1012mm","Unit STRT45d\"\"MC\"\"Book2280x823x584 Blk","Shelf Wire 1258x467x22mm","Frame H D/S 1804x1078mm","Frame H D/S 1518x1078mm","Shelf Curved 1255x597mm","Beam Adjustable1282mm","Panel Side W/Clip Top 321x1010mm","Panel Side W/Clip Botom 531x1250mm","Frame H Single 2280x539mm","Beam End 1258x115mm","Kit Punched Panel W/joiners 1271x682mm","Bottle Holder Small - Fine Wine","Lock Box Double Shelf 1245x755 Black","Unit End FGT 1078Xx400Dx1804H Black","Kit End Shelf Curve Suit FGD NarrowUnits","Bottle Holder Acrylic DM","Unit START B PrWine 2280x1832x768 Blk","Unit ADD B PrWine 2280x1832x768 Blk","Kit B Unit Bottle Holder 9 Column 110mm","Kit B Unit Bottle Holder 9 Column 65+95m","Board Beer Signage U Shape DM","Sign FG Units adjT-Bar Hang Panel Blk","Roll Cage Liquor Security","Kit Liquor C/Out Wire Basket&Bag Holder","Spring Loaded Trolley","A4 Double Sided Sign Holder","Sign Category Variable 170x450mm BWS","Sign Push/Pull Door Decal BWS Only \\(2\\)","Sign Cold Beer Bulkhead Decal BWS Only","Sign Conditions of Entry BWS Only","Sign Trading Hours D/Sided BWS Only","Sign Double Sided I'm New Header","Fin I'm New 135x800mm","Magnetic Holder Black DM","Baskets Hand \"Dan Murphy's\" Black","Clip Ticket Holder Pack of 100","A3 Signage Frames Blk","A4 Signage Frames Blk","Telescopic Stands","Magnetic Frame Clip","Sign Refund Policy 345x520","Sign Guarantee D/S 1000x1000","Poster A3 10% Off - DM","Talker Shelf 10% Off - DM","Sign Wall Capping Hook Over 1283x237","Sign External Freezer Ice Decal","Sign A1 Clear Insert Display Sheet","Sign Tasting Table 1640x480","Data Stripping Insert Thin Insert1260x28","Sign Langtons Fin 250x500","Data Stripping Insert Langtons 1835x38","Sign Magnetic Dairy Bay Cap Mix & Match","Sign Electrostatic Mix&Match 650x100","Sign Magnetic Here's Proof 850x310","Holder Price A4 Acrylic Pk 25","Fin Cocktail Bay 150x1150","Panel Centre Cocktail Bay 500x1150","Data Stripping Cocktail Bay 1260x38mm","Kit X2 Sign Cocktail Bay 1260x140","Gripper Self-Adhesive Rigid","Board Sales Promotion DM 1800x1200","Board Pin General Notices DM 1450x1200","BoardPin Company Objectives DM 1000x1200","Board Pin Staff News DM 760x1200","Board Emergency Evacuation DM 990x660","Board Pin Safety Notice DM 1448x1220","Sign FGD End Aisle Signage Panel Only RH","Sign FGT End Aisle Signage Tall Panel","Sign FGD End Aisle Signage Panel Only LH","Sign FGT End Desc Single Insert","FGD Large Vertical Fin S/Sign 590Dx88W","Sticker 'Do Not Stand On Ledge' 30x400 D","Sign 24-7 Window Decal 750x750mm","Sign Sec Camera Folded 102x340 D/M Blk","Sign Conditions of Entry D/S 700x700","Sign Video Surveillance 200x300 D/M Blk","Sign Light Box Small Insert 1265x440","Sign Light Box Large Insert 1775x440","Data Stripping Fat Insert 1260x38","Data Stripping S/Fat Insert 1293x100","Sign Wall Capping Hook Over 1283x440","1 Bay Wall Wanzl","2 Bay Wall Wanzl","3 Bay Wall Wanzl")
  , MaterialMaster_BWS_301_400 = c("4 Bay Wall Wanzl","5 Bay Wall Wanzl","6 Bay Wall Wanzl","Sign Category Fridge Door Pelmet 788x175","Signage Wall Capping w/mag holders","Sign Category Fine Wine 750x150","Sign Trading Hours D/S 500x700","Sign FGT End Aisle Signage Tall Panel Bl","Sign Category FGD D/Sided Fin 88x650","Kit Sign FineWineA3&A4 Frame&Stand Blk","Signage Wall Capping w/mag holders Black","Data Stripping Mix & Match 1260x38","Sign FGT End Magnetic AisleBoard 357x620","Sign FGD End Mag Aisle Board LH 242x477","Sign FGD End Mag Aisle Board RH 242x477","Sign Category FGT End Magnetic 70x330","Sign Category FGD Magnetic 50x236","Sign Category Horizontal Beer 1280x380","Sign Category Horizontal Wine 1280x300","Sign PVC Wanzl & Stand 1800x450","Sign Category Island Dairy BayM 1250x220","Decal Island Dairy M Bay","Decal Island Dairy Bay M End","Sign Category Back of Aisle Mag 420x297","Sign Board Back of Aisle Mag 840x297","Sign Magnetic Dairy Bay Capping 1260x140","Sign Alphabet Sticker","Data Strip Cladding TimberTrim Low Prof","End Capping Spine TimberTrim Low Profile","Mid Capping Spine TimberTrim Low Profile","Low Profile Timber End Panel 1120\\(W\\)mm","Display Unit Slanted Timber Low Profile","End Fixture Timber Low Profile 1360Wmm","End Fixture Timber Low Profile 1430Wmm","Low Profile Timber End Panel - 1360\\(W\\)mm","End Fixture Timber L/P w/Door 1120\\(W\\)mm","End Fixture Timber L/P no/Door 1120\\(W\\)mm","End Fixture Timber L/P no/Door 1360\\(W\\)mm","End Fixture Timber L/P no/Door 1430\\(W\\)mm","Low Profile Timber End Panel - 1430\\(W\\)mm","Open End Fixture LP 1120\\(W\\)mm","Open End Fixture LP 1360\\(W\\)mm","Open End Fixture LP 1430\\(W\\)mm","G Counter Freestanding LH 3000Wx2600Dmm","G Counter Freestanding RH 3000Wx2600Dmm","G Counter Wall LH 3000Wx2400Dmm","G Counter Wall RH 3000Wx2400Dmm","G Counter Wall LH 3000Wx2250Dmm","G Counter Wall RH 3000Wx2250Dmm","G Counter Rear Low Ht Display Hob 900Wmm","G Counter Slat Wall 1000Wx1420Hmm","G Counter Timber Counter Stool","Counter Wall Front Unit - Retrofit","Data Strip Cladding TimberTrim HP 1135mm","Vertical Timber Trim High Profile","Data Kickplate Timber Trim HP 1135mm","End Panel with OverheadCabTimberTrimHP","Cabinet Overhead Timber HP 1135mm","DataStrip CladdingimberTrim HP 535mm","DataKickplate TimberTrim HP 535mm","Cabinet Overhead Timber HP 550mm","Gantry Board 1800\\(W\\) x 295\\(H\\) mm","Slat Wall Shelf Kit","Slat Wall Hook With Ticket Face","Column Cladding 1200 High","Column Cladding 2400 High","Drawer Unit HP 1135mm","Drawer Unit HP 535mm","Low Profile Timber End Panel - 700\\(W\\)mm","Mobile Merchandise Trolley","Snackfoods Shelf","Folding Tasting Table","Drawer Unit LP 915mm","Kit Spill Dan Murphy Complete","Kit Spill Dan Murphy Absorbent 22Ltr","Rack Mount Bracket x2 L/R 12 Bottles M","Cabinet Pro Loc Conversion Bracket&Screw","Rack Mount Bracket x2 Set L/R 17","Cabinet Pro Loc Locks","Rack Mount Bracket x2 L/R 12 Bottles MX","Rack 12 Bottle Position Lock System","Rack 17 Bottle Position Lock System","Lock Box Triple Shelf 1245x1200 LED","Box Lock Double Shelf 1245x754.6 LED","Box Lock Single Shelf 1245x360x360 LED","Scanner Fingerprint Office Input Log","Scanner Fingerprint 4 Rack Operation","Traffic Cones 700mm 3.2KG Orange","SafetyCone 1m Reflective wHook&Sign Hold","Yellow Chain Supplied in 2x 5m Lengths","Safety Sign - Forklifts in Use","Safety Sign - Speed Limit 5","Ladder Mobile Platform 1420mm","Ladder Mobile Platform Step 850mm","Glasswasher Rhima DS-45T","StandTwin Liquor Service&Screw NCRMonito","Shelf Wire 1258x467x22mm","Roll Cage Liquor Security","Kit Liquor C/Out Wire Basket&Bag Holder","Trolley Rotacaster Hand Truck BWS")
  , MaterialMaster_BWS_401_500 = c("Data Strip Curved Wing Stack 395x99 Pk40","Data Strip \"Radius\" 1293x108mm Black","Data Strip 39mm x 1250mm Black Ambient","Holder Sign for Data Strip Clear DM","Data Strip Curved Wood Box 220x99 Pk28","Data Strip Vintec Kit-Qty 50","Data Strip Radius 1293x99mm-Inserts","Data Strip 39mm x 1250mm Black Chilled","Data Strip 39mm x 1250mm Clear Ambient","Stand A3 Tobacco Pricing Display DM","Lock Box Trading Hour DM","Base Metal - Magazine Stand DM","Display - Magazine Stand DM","Rack Hanging Magazine Stand DM","U-Rail 125mm with Adhesive Transparent","Frame A5 Landscape Black","A3 Acrylic PVC Sleeves","A4 Acrylic PVC Sleeves","Board Feature End Black 200x16x948mm 1A","Board Black 200x16x1846mm 2A","Board Black 200x16x1826mm 3A","Frame End FGT 1804x400mm","Kit FGT Shelf1005x332x22&Beam End 1012mm","Unit STRT45d\"\"MC\"\"Book2280x823x584 Blk","Shelf Wire 1258x467x22mm","Frame H D/S 1804x1078mm","Frame H D/S 1518x1078mm","Shelf Curved 1255x597mm","Beam Adjustable1282mm","Panel Side W/Clip Top 321x1010mm","Panel Side W/Clip Botom 531x1250mm","Frame H Single 2280x539mm","Beam End 1258x115mm","Kit Punched Panel W/joiners 1271x682mm","Bottle Holder Small - Fine Wine","Lock Box Double Shelf 1245x755 Black","Kit End Shelf Curve Suit FGD NarrowUnits","Bottle Holder Acrylic DM","Unit START B PrWine 2280x1832x768 Blk","Unit ADD B PrWine 2280x1832x768 Blk","Kit B Unit Bottle Holder 9 Column 110mm","Kit B Unit Bottle Holder 9 Column 65+95m","Board Beer Signage U Shape DM","Sign FG Units adjT-Bar Hang Panel Blk","Shelf Wire Mid 915x600mm Sat Black","Shelf Wire Base 915x600mm S/Black Slant","Bracket 600mm Sat Black","Shelf Metal 610x505mm Satin Black","Post L 1500x535mm Satin Black","Panel Plain 450x902mm Satin Black","Board Feature End Black 200x16x948mm 1A","Board Black 200x16x1846mm 2A","Board Black 200x16x1826mm 3A","Board Black 200x16x913mm 4A","Bracket 470mm 6 Notch Satin Black","Post T 1469x470mm","Post L 1469x400mm Narrow","Shelf Base 913x400mm Narrow","Bracket ADJ 355mm Satin Black","Beam Light Duty 1135x45mm Satin Black","Beam Light Duty 535x45mm Satin Black","Post H Frame 2400x505mm Satin Black","Post H Frame 1800x505mm Satin Black","Shelf Melamine Black 1166x42mm","Shelf Melamine Black 565x430mm","Shelf Metal 1166x505mm Satin Black","Panel End Metal Satin Black","Holder Sign Low Profile","Table Staff Lunch/Train 1200x750x755","Pallet Mini Beer PK20","Pallet Mini Beer \\(Small Grey\\) PK12","Trolley Mini Beer","Fridge Bar Staff 115lt Fisher & Paykel","Phone Cordless Uniden","TV Hisense 50 N4 LED/LCD + Wall Bracket","Tape Safety Coolroom 15metres","Decals Security Height Indicator Set 2 B","Kit Magnetic Pricing Posterboard BWS","Kit Magnetic Numbers AO Boards","Sign Security Notice 400x300mm DM","Holder Book Acrylic","Frame Double 930x340mm Black","Frame Single 475x340mm Black","Impulse Rack w/Slatwall 1284x1454mm","Merchandiser Free Standing Case DM","Box Lock Single Shelf 1245x530 Black","Tensa Barrier Retractor","Trolley Mobile Wine Tasting Unit","Frame H Single Mid Height 1340 x 539mm","Panel Side Mid Height 1320 x 471mm","Panel Back Mid Height 1320 x 660mm","Frames & Stands A1","Kit Here's Proof Board","Sign Here's Proof Skin","Shelf Lay Down FGD/FGT 195x1258x467","Barrel Wine Half TS&B","Panel Side W/Clip Top 321x1010mm","Panel Side W/Clip Botom 531x1250mm","Frame H Single 2280x539mm","Lock Box Double Shelf 1245x755 Black")
  , MaterialMaster_BWS_501_600 = c("Stand Half Wine Barrel Std 905mm Black","Frame End FGD 1518x400mm","Shelf FGD End Curve Top 1078x260mm","Shelf FGD End Curve Bottom 1078x400mm","Frame FGD Extension Double 1078x624x32","Frame FGD Extension Single 571x624x32","WallDataExpSlideDoorMetalRH741x905Black","WallDataExpSlideDoorMetalLH741x905Black","A1 Skins","Beer Board Rotating Replacement Sleeve","Beer Board Rotating Single","Beer Board Rotating Double","C/O L Shape Express RH Blk","C/O L Shape Express LH Blk","Counter Unit Click & Collect DM","C/O Exp I/Safe M/Blk1087x757x900LH Metal","C/O Exp I/Safe M/Blk1087x757x900RH Metal","C/O DoubleI/Safe M/Blk2600x2100x900Metal","C/OSingleI/SafeM/Blk1416x2100x900LHMetal","C/OSingleI/SafeM/Blk1416x2100x900RHMetal","Counter Cigarette M/Blk950x605x1065Metal","Counter Miniature M/Blk950x605x1065Metal","Click&Collect Unit Blk1728x600x1050Metal","Unit Open WBag Pack 477Wx628Dx868H G/Oak","Corner Void 520Wx628Dx868H G/Oak LH","Unit Storage Return 878Wx480Dx868H G/Oak","Unit Cash Vac 520Wx483Dx868H G/Oak","Unit CigDraw Liquor 918Wx430Dx868H G/Oak","Unit High GI/Shelf 918Wx280Dx1332H G/Oak","Unit High Circle 918Wx280Dx1332H G/Oak","Unit Low GlassShelf 918Wx280Dx532H G/Oak","Unit Corner Void 520Wx628Dx868H G/Oak RH","Unit Reg Liquor 600Wx628Dx868H G/Oak","Benchtop Front 32Hx898DxLength to suitRH","Unit Front Infill 600Wx628Dx868H G/OAK","Benchtop Front 32Hx898DxLength to suitLH","Intellisafe BWS - Tank","Beam Light Duty 1135x45mm Satin Black","Post H Frame 1800x505mm Satin Black","Shelf Metal 1166x505mm Satin Black","Frame End FGT 1804x400mm","Kit FGT Shelf1005x332x22&Beam End 1012mm","Unit STRT45d\"\"MC\"\"Book2280x823x584 Blk","Shelf Lay Down FGD/FGT 195x1258x467","Beam Stabilising 1258mm","Shelf Wire 1258x467x22mm","Frame H D/S 1804x1078mm","Frame H D/S 1518x1078mm","Shelf Curved 1255x597mm","Beam Adjustable1282mm","Panel Side W/Clip Top 321x1010mm","Panel Side W/Clip Botom 531x1250mm","Frame H Single 2280x539mm","Beam End 1258x115mm","Kit Punched Panel W/joiners 1271x682mm","Bottle Holder Small - Fine Wine","Lock Box Double Shelf 1245x755 Black","Kit End Shelf Curve Suit FGD NarrowUnits","Bottle Holder Acrylic DM","Beam Bottom FGD/FGT 1258mm","Beam Bottom Adjustable FGD/FGT 1258mm","Bumper Rail FGD/FGT 1285mm","Unit START B PrWine 2280x1832x768 Blk","Unit ADD B PrWine 2280x1832x768 Blk","Kit B Unit Bottle Holder 9 Column 110mm","Kit B Unit Bottle Holder 9 Column 65+95m","Board Beer Signage U Shape DM","Sign FG Units adjT-Bar Hang Panel Blk","Cabinet Bottle RH Black Vintec 170","Trolley Top Deck 1200x600","Stand Mobile Basket DM","Unit Wall Wanzl 1800 1 Bay Start","Unit Wall Wanzl 1800 1 Bay Add","Display Unit 2Bay Round End3600x1000 Blk","Box Wooden for Wanzl Unit 460x330x190mm","Kit Llexan Security Bottle Cap & Tags DM","Detacher Magnetic Countertop Tool")
)

for(i in 1:length(MaterialMaster_BWS_LIST)){
  x <- MaterialMaster_BWS_LIST[[i]]
  npiped <- paste0(x, collapse="|")
  which_no_rule_yet <- which(FileData_ALL$Rule_Food == '' & FileData_ALL$Rule_BWS == '') # this is not a typo if Rule_Food is not blank then Rule_BWS should be blank.
  if(length(which_no_rule_yet)<1){break}
  which_match <- grep(npiped, FileData_ALL$Asset_description[which_no_rule_yet], ignore.case = TRUE)
  idx_match <- which_no_rule_yet[which_match]
  FileData_ALL$Rule_BWS[idx_match]  <- 'BWS'
  print(i)
}

#Rule 6.3 - WBS starting with G = Emergency
FileData_ALL$Rule_Food[intersect(
  grep("^G", FileData_ALL$WBS_element,ignore.case = TRUE)
  , which(FileData_ALL$Rule_BWS == '' & FileData_ALL$Rule_Food == '')
  )]  <- 'Food'

#### Check that results are what we expect
sum(FileData_ALL$Rule_BWS != '' & FileData_ALL$Rule_Food != '')

table(FileData_ALL$Rule_Food[grep("^G", FileData_ALL$WBS_element,ignore.case = TRUE)]
      , FileData_ALL$Rule_BWS[grep("^G", FileData_ALL$WBS_element,ignore.case = TRUE)])

#Rule 7 - All else  = Shared
#===============================================================================================
FileData_ALL$Rule_Shared <- ifelse(FileData_ALL$Rule_BWS == 'BWS' | FileData_ALL$Rule_Food == 'Food', '', 'Shared')

FileData_ALL$Rule_Final <- ifelse(FileData_ALL$Rule_BWS == 'BWS', 'BWS', 
                                  ifelse(FileData_ALL$Rule_Food == 'Food', 'Food',
                                         ifelse(FileData_ALL$Rule_Shared == 'Shared', 'Shared',''
                                         )))
# 
# setwd("U:\\Projects\\Backup_C_Projects_20191021\\ED - Swan Data\\20200313_0831_AssetLoad\\DataFromJason\\Output")

FileData_ALL <- FileData_ALL %>% 
  dplyr::filter(!is.na(Asset))

saveRDS(FileData_ALL,"FileData_ALL.rds")
write.csv(FileData_ALL, "FileData_ALL.csv", row.names = F, quote = T)

if(!is.na(pc_to_run) %>% all()){
  for(mcc in pc_to_run){
    OutputData <- FileData_ALL %>%
      dplyr::filter(Cost_Ct == mcc)
    
    data.table::fwrite(OutputData,paste0(mcc,".csv"))
  }
}

# Data summary
data_summary <- FileData_ALL %>%
  dplyr::group_by(Cost_Ct, Year, Clas, Rule_Final, Rule_FormatType) %>%
  dplyr::summarise(
    APC_FY_start = sum(APC_FY_start, na.rm=TRUE)
    , Curr_bk_val = sum(Curr_bk_val, na.rm=TRUE)
    , Dep_FY_start = sum(Dep_FY_start, na.rm=TRUE)
    , Bk_val_FY_strt = sum(Bk_val_FY_strt, na.rm=TRUE)
    , Acquisition = sum(Acquisition, na.rm=TRUE)
    , Dep_for_8MTHS = sum(Dep_for_8MTHS, na.rm=TRUE)
    , Retirement = sum(Retirement, na.rm=TRUE)
    , Dep_retir = sum(Dep_retir, na.rm=TRUE)
    , IntraCoy_Trf = sum(IntraCoy_Trf, na.rm=TRUE)
    , IntraCoy_Dep = sum(IntraCoy_Dep, na.rm=TRUE)
    , InterCoy_Trf = sum(InterCoy_Trf, na.rm=TRUE)
    , InterCoy_Dep = sum(InterCoy_Dep, na.rm=TRUE)
    , Reval_Depn = sum(Reval_Depn, na.rm=TRUE)
    , Reval_APC = sum(Reval_APC, na.rm=TRUE)
    , Current_APC = sum(Current_APC, na.rm=TRUE)
    , Accumul_dep = sum(Accumul_dep, na.rm=TRUE)
  )
data.table::fwrite(data_summary,"data_summary.csv")



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



[1] TRUE

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6


[1] 0

      
           BWS
         0   3
  Food 154   0

`summarise()` has grouped output by 'Cost_Ct', 'Year', 'Clas', 'Rule_Final'. You can override using the `.groups` argument.



In [51]:
########## Data_Prep.R

library(tidyverse)
library(readxl)

## PURPOSE:
## produce a csv ready for SAP ingestion for December 2019 swan asset load.

# # just these 3 files as confirmed by Jason Bennett 20191125 14:06
# "F19 Conv FAR detailed Z056 all Classes run 12.07.2019.csv"
# "F19 Supermarkets FAR detailed Z056 All except Class 3000 run 12.07.2019.csv"
# "F19 Supermarkets FAR detailed Z056 Class 3000 run 12.07.2019.csv"


## Import data - above files already run and saved to here with year == 'FY19'
#setwd("U:\\Projects\\Backup_C_Projects_20191021\\ED - Swan Data\\20200313_0831_AssetLoad\\DataFromJason\\Output")
dat <- readRDS("FileData_ALL.rds")

## import Jasons deactivated lists -the list of asset number, SN pairs to delete for SAP load.
#setwd("U:\\Projects\\Backup_C_Projects_20191021\\ED - Swan Data\\data_deactivated_assets_fy19")
#setwd("data_deactivated_assets_fy19")

mcolTypes <- c("numeric", "numeric", "numeric", "date")
mcolNames <- c("Asset", "SN", "CoCd", "Deact_Date")

fileList <- list()
#for( mfile in list.files()){
for( mfile in data_deactivated_asset_files){  
  msheets <- readxl::excel_sheets(mfile)
  for(msheet in msheets){
    listName <- length(fileList)+1
    fileList[[listName]] <- readxl::read_xlsx(mfile
                                              , sheet=msheet
                                              , skip = 1
                                              , col_types = mcolTypes
                                              , col_names = mcolNames
    )
    
  }
  print(paste0(mfile, " complete"))
}

df_deactivated <- dplyr::bind_rows(fileList)

# change from dateTime to date only
df_deactivated$Deact_Date <- df_deactivated$Deact_Date %>% as.Date(.)

# check for duplicates:
# df_deactivated %>% 
#   dplyr::group_by(Asset, SN) %>% 
#   dplyr::summarise(nn=n_distinct(CoCd, Deact_Date)) %>% 
#   dplyr::filter(nn>1)

# there were some duplicates to avoid joining many to one just take max deact date by ...
df_deactivated_clean <- df_deactivated %>%
  dplyr::group_by(Asset, SN, CoCd) %>%
  dplyr::summarise(Deact_Date = max(Deact_Date, na.rm=T))

# check for duplicates:
# df_deactivated_clean %>%
#   dplyr::group_by(Asset, SN) %>%
#   dplyr::summarise(nn=n_distinct(CoCd, Deact_Date)) %>%
#   dplyr::filter(nn>1)

# join on deact date
dat_joined <- dat %>%
  dplyr::left_join(
    df_deactivated_clean
    , by=c("Asset", "SN", "CoCode"="CoCd")
  )

nrow(dat_joined)==nrow(dat)

mean(is.na(dat_joined$Deact_Date))
sum(!is.na(dat_joined$Deact_Date))

# now calc the BWS allocation of shared:
# from Bonika's sheet 'Materials Master & Store List' (https://docs.google.com/spreadsheets/d/1pbuJ84AVMzRxIdL2_Zmb8JwbFOF5HeU5Bst63Z4r2xc/edit?usp=sharing)
# but downloaded locally, or just created here:

allocations_method <- tibble(
  Clas=c(1000, 1100, 1200, 2500, 3400, 3000, 7600, 3200, 4000, 4100, 4200, 7100, 2300, 2000, 3300, 7000, 9000, 9100, 7300, 3100, 8100, 8000, 2200, 2400, 2700, 7200, 7601, 7602, 6000, 6100, 7900, 7700, 7400, 3500)
  , Project=c(NA, NA, 'BA', NA, 'BA', NA, 'BA', 'TA', 'TA', 'TA', 'TA', NA, NA, NA, 'BA', NA, NA, NA, NA, NA, NA, NA, NA, NA, 'BA', NA, 'BA', 'BA', 'TA', 'TA', NA, NA, NA, NA)
  , New=c(NA, NA, 'BA', NA, 'BA', NA, 'BA', 'TA', 'TA', 'TA', 'TA', 'TA', NA, NA, 'BA', 'BA', NA, NA, NA, NA, NA, NA, NA, NA, 'BA', 'BA', 'BA', 'BA', NA, NA, NA, NA, NA, NA)
  , `WBS Zero`=c(NA, NA, 'BA', NA, 'BA', NA, 'BA', 'BA', 'BA', 'BA', 'BA', NA, 'BA', 'BA', 'BA', NA, NA, NA, 'BA', 'BA', 'BA', 'BA', NA, NA, 'BA', NA, 'BA', 'BA', NA, NA, NA, NA, NA, NA)
)
# new column is actually Renewals, Extensions & New Stores so make these columns before going long
allocations_method$Extention <- allocations_method$New
allocations_method$Renewal <- allocations_method$New

# check that all unique format types are in columns of allocations_method
setdiff(unique(dat_joined$Rule_FormatType), colnames(allocations_method))

allocations_method_wide <- reshape2::melt(allocations_method
                                          , id.vars=c("Clas")
                                          , variable.name = "Rule_FormatType"
                                          , value.name = "Allocation_Type"
) %>%
  dplyr::filter(!is.na(Allocation_Type))

# now join on allocation type
nrow0 <- nrow(dat_joined)
dat_joined <- dat_joined %>%
  dplyr::left_join(
    allocations_method_wide
    , by=c("Clas", "Rule_FormatType")
  )
nrow1 <- nrow(dat_joined)

if(nrow0 != nrow1){stop("Join created additional records!")}

# # Now get and join allocation %s.
# from Bonika's sheet 'Materials Master & Store List' (https://docs.google.com/spreadsheets/d/1pbuJ84AVMzRxIdL2_Zmb8JwbFOF5HeU5Bst63Z4r2xc/edit?usp=sharing)
## but downloaded locally


#setwd("U:\\Projects\\Backup_C_Projects_20191021\\ED - Swan Data\\Store List & Allocation Rate")

file2Imp <- storeList_and_allocationRate

range2String <- function(range, col){
  #'3:561','I:J' => 'I3:J561'
  s_ <- unlist( strsplit(range,":") )
  c_ <- unlist( strsplit(col,":") )
  return(paste0(c_[1],s_[1],':',c_[2],s_[2]))
}

aRange <- range2String(storeList_and_allocationRateRange,'A:A')
iRange <- range2String(storeList_and_allocationRateRange,'I:J')

store_allocations <- readxl::read_xlsx(file2Imp
                                       , sheet="Sheet1"
                                       , range = aRange #"A3:A561"
                                       , col_types = "numeric"
                                       , col_names = "Cost_Ct"
) %>%
  dplyr::bind_cols(
    readxl::read_xlsx(file2Imp
                      , sheet="Sheet1"
                      , range = iRange #"I3:J561"
                      , col_types = c("numeric", "numeric")
                      , col_names = c("BA", "TA")
    )
  )

store_allocations_long <- reshape2::melt(store_allocations
                                         , id.vars=c("Cost_Ct")
                                         , variable.name = "Allocation_Type"
                                         , value.name = "Allocation_Perc")

# now join on allocation rates
nrow0 <- nrow(dat_joined)
dat_joined <- dat_joined %>%
  dplyr::mutate(Cost_Ct=as.numeric(as.character(Cost_Ct))) %>% # convert to numeric for join
  dplyr::left_join(
    store_allocations_long
    , by=c("Cost_Ct", "Allocation_Type")
  )
nrow1 <- nrow(dat_joined)
if(nrow0 != nrow1){stop("Join created additional records!")}

# now allocate rules.
dat_joined$bws_perc <- ifelse(dat_joined$Rule_Final == "BWS", 1
                              , ifelse(dat_joined$Rule_Final == "Food", 0,
                                       dat_joined$Allocation_Perc
                              ))
dat_joined$bws_perc[is.na(dat_joined$bws_perc)] <- 0

dat_joined$bws_dollars <- dat_joined$Current_APC*dat_joined$bws_perc
dat_joined$bws_dollars[is.na(dat_joined$bws_dollars)] <- 0
dat_joined$non_bws_dollars <- dat_joined$Current_APC-dat_joined$bws_dollars


# save files:
#setwd("U:\\Projects\\Backup_C_Projects_20191021\\ED - Swan Data\\20200313_0831_AssetLoad\\Output")

readr::write_csv(dat_joined %>% dplyr::filter(Cost_Ct == 3354), "3354.csv")
readr::write_csv(dat_joined %>% dplyr::filter(Cost_Ct == 1004), "1004.csv")
readr::write_csv(dat_joined %>% dplyr::filter(Cost_Ct == 1086), "1086.csv")
saveRDS(dat_joined, "dat_joined.rds")
#readr::write_csv(dat_joined, "dat_joined.csv")

## now create template:
# template from Abhy Rao to Jason Corderoy 25 Nov 2019 14:40
# rows to include in template.
which_rows_delete <- c(
  which(dat_joined$bws_dollars == 0) # if BWS not getting any the ignore
  , which(is.na(dat_joined$bws_dollars)) # if BWS not getting any the ignore
  , which(!is.na(dat_joined$Deact_Date)) # We can ignore deactivated assets for this purpose
) %>%
  unique()

SAP_Data <- dat_joined[-which_rows_delete,]

final_sap_file <- data.frame(
  ASSET_NUM=SAP_Data$Asset
  , SUB_AST_NUM=SAP_Data$SN
  , DOC_DATE=rep(NA, nrow(SAP_Data))
  , POST_DATE=rep(NA, nrow(SAP_Data))
  , AST_VAL_DT=rep(NA, nrow(SAP_Data))
  , AST_CLASS=SAP_Data$Clas
  , COST_CTR=SAP_Data$Cost_Ct
  , RESP_CCTR=rep(NA, nrow(SAP_Data))
  , ZORD_42=rep(NA, nrow(SAP_Data))
  , ZAMOUNT=round(SAP_Data$bws_dollars, 2)
)

# avoid outputting scientific notation
options(scipen=999)

#setwd("U:\\Projects\\Backup_C_Projects_20191021\\ED - Swan Data\\20200313_0831_AssetLoad\\Output")
write.csv(final_sap_file
          , "final_sap_file.csv"
          , row.names=F
          , na = ''
          , quote = F)
saveRDS(final_sap_file, 'final_sap_file.rds')

write.csv(SAP_Data
          , "SAP_Data_Wide.csv"
          , row.names=F
          , na = ''
          , quote = T # put quote on else csv opens weird - there are commas in asset description!!
          )
saveRDS(SAP_Data, 'SAP_Data_Wide.rds')




[1] "F19 BOOK Disposal check AUD Co codes 1000 project SWAN list.xlsx complete"
[1] "F19 BOOK Disposal check AUD Co codes 1010-1999 project SWAN list.xlsx complete"
[1] "F19 but F20 BOOK Disposal check AUD Co codes 1000-1910 in QTRS SWAN to 26.11.19 .xlsx complete"


`summarise()` has grouped output by 'Asset', 'SN'. You can override using the `.groups` argument.



[1] TRUE

[1] 1

[1] 0

character(0)

In [52]:
##### save entire environment
save.image('env.RData')

In [53]:
##### Zip it all!!!!!!!!
zip(zipfile = 'zipped_all_files', files = list.files())